## with PreprocessMask

In [1]:
import numpy as np
import cv2
import os
from tqdm import tqdm as tqdm
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.losses import binary_crossentropy
from keras import backend as K
import segmentation_models as sm
# from keras.utils import to_categorical
from tensorflow.keras.utils import to_categorical
from tensorflow.python.keras.utils import generic_utils
from segmentation_models.losses import CategoricalFocalLoss
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.utils import to_categorical

from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau,ModelCheckpoint
# from keras.optimizers import Adam,SGD
from keras.losses import BinaryCrossentropy
from keras.metrics import MeanIoU
from glob import glob
import tensorflow_model_optimization as tfmot
from tensorflow_model_optimization.quantization.keras import QuantizeConfig
import tensorflow_model_optimization as tfmot

2024-07-01 12:33:00.668671: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-01 12:33:00.779345: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/agrograde/.local/lib/python3.8/site-packages/cv2/../../lib64:/opt/ros/noetic/lib
2024-07-01 12:33:00.779366: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-07-01 12:33:00.803674: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS 

Segmentation Models: using `keras` framework.


In [2]:
 print('Num GPUs Available: ', len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  0


2024-07-01 12:33:02.605828: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
2024-07-01 12:33:02.605860: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: agrograde-System-Product-Name
2024-07-01 12:33:02.605866: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: agrograde-System-Product-Name
2024-07-01 12:33:02.605980: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 535.183.1
2024-07-01 12:33:02.606003: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 535.171.4
2024-07-01 12:33:02.606009: E tensorflow/stream_executor/cuda/cuda_diagnostics.cc:313] kernel version 535.171.4 does not match DSO version 535.183.1 -- cannot find working devices in this configuration


In [3]:

def preprocessMasks(mask,height,width):
    mask_resized = cv2.threshold(cv2.resize(mask, (height,width)), 50, 1, cv2.THRESH_BINARY)[1]
    mask_data = np.zeros((height,width,2))
    
    for i in range(height):
        for j in range(width):


            #for segmentation mask
            if mask_resized[i,j]> 0:
                mask_data[i,j,1] = 1
            else:
                mask_data[i,j,0] = 1
                
    return mask_data       #output from the function(height, width, 2)

In [4]:
def load_data(path):
    images = sorted(glob(os.path.join(image_path, 'images/*')))
    sprout = sorted(glob(os.path.join(image_path, 'Sprout/*')))
    peeled = sorted(glob(os.path.join(image_path, 'Peeled/*')))
    rotten = sorted(glob(os.path.join(image_path, 'Rotten/*')))
    double = sorted(glob(os.path.join(image_path, 'Double/*')))
    background = sorted(glob(os.path.join(image_path, 'Background/*')))
    return images,sprout,peeled,rotten,double,background
    

In [5]:
image_path = '/home/agrograde/Desktop/4th_mar/test_image_path'

In [6]:
# image_path = '/home/agrograde/Desktop/4th_mar/test_image_path'

In [7]:
images,sprout,peeled,rotten,double,background = load_data(image_path)

In [8]:
print(len(images))
print(len(sprout))
print(len(peeled))
print(len(rotten))
print(len(double))
print(len(background))

30
30
30
30
30
30


In [9]:
def read_image(path):
    
    x = cv2.imread(path)
    x = cv2.resize(x, (224, 224))
    x = x.astype(np.float32)
    return x
    

In [10]:
def read_mask(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = preprocessMasks(x, 224, 224)
    x = x.astype(np.float32)
    return x
    
    

In [11]:
# def visualize_sample(image_path, mask_paths):
#     image = read_image(image_path)
#     masks = [read_mask(mask_path)[:, :, 1] for mask_path in mask_paths]
    
#     plt.figure(figsize=(20, 10))
    
#     plt.subplot(1, 6, 1)
#     plt.imshow(image.astype(np.uint8))
#     plt.title('Image')
#     plt.axis('off')
    
#     titles = ['Sprout Mask', 'Peeled Mask', 'Rotten Mask', 'Double Mask', 'Background Mask']
#     for i, mask in enumerate(masks):
#         plt.subplot(1, 6, i + 2)
#         plt.imshow(mask, cmap='gray')
#         plt.title(titles[i])
#         plt.axis('off')
    
#     plt.tight_layout()
#     plt.show()

# # Visualize a few samples to ensure correct pairing
# for i in range(5):
#     visualize_sample(images[i], [sprout[i], peeled[i], rotten[i], double[i], background[i]])


In [12]:
# x = read_image(images[0])
# y1 = read_mask(sprout[0])
# y2 = read_mask(peeled[0])
# y3 = read_mask(rotten[0])
# y4 = read_mask(double[0])
# y5 = read_mask(background[0])

In [13]:
# print(x.shape,y1.shape,y2.shape,y3.shape,y4.shape,y5.shape)

In [14]:
# def preprocess(x,y1,y2,y3,y4,y5):
#     def f(x,y1,y2,y3,y4,y5):
#         x = x.decode()
#         y1 = y1.decode()
#         y2 = y2.decode()
#         y3 = y3.decode()
#         y4 = y4.decode()
#         y5 = y5.decode()
        
#         x = read_image(x)
#         y1 = read_mask(y1)
#         y2 = read_mask(y2)
#         y3 = read_mask(y3)
#         y4 = read_mask(y4)
#         y5 = read_mask(y5)
        
#         return x,y1,y2,y3,y4,y5
    
#     images, sprout, peeled, rotten,double,background = tf.numpy_function(f, [x,y1,y2,y3,y4,y5], [tf.float32, tf.float32,tf.float32,tf.float32,tf.float32,tf.float32])
#     images.set_shape([224, 224, 3])
#     sprout.set_shape([224, 224, 2])
#     peeled.set_shape([224, 224, 2])
#     rotten.set_shape([224, 224, 2])
#     double.set_shape([224, 224, 2])
#     background.set_shape([224, 224, 2])

#     return images, sprout, peeled, rotten,double, background


In [15]:
def preprocess(x, y1, y2, y3, y4, y5):
    def f(x, y1, y2, y3, y4, y5):
        x = x.decode()
        y1 = y1.decode()
        y2 = y2.decode()
        y3 = y3.decode()
        y4 = y4.decode()
        y5 = y5.decode()
        
        x = read_image(x)
        y1 = read_mask(y1)
        y2 = read_mask(y2)
        y3 = read_mask(y3)
        y4 = read_mask(y4)
        y5 = read_mask(y5)
        
        return x, y1, y2, y3, y4, y5
    
    images, sprout, peeled, rotten, double, background = tf.numpy_function(
        f, [x, y1, y2, y3, y4, y5], 
        [tf.float32, tf.float32, tf.float32, tf.float32, tf.float32, tf.float32]
    )
    
    images.set_shape([224, 224, 3])
    sprout.set_shape([224, 224, 2])
    peeled.set_shape([224, 224, 2])
    rotten.set_shape([224, 224, 2])
    double.set_shape([224, 224, 2])
    background.set_shape([224, 224, 2])

    return images, sprout, peeled, rotten, double, background

In [16]:
def tf_dataset(x, y1, y2, y3, y4, y5, batch_size, train_split=0.8, val_split=0.1):
    dataset = tf.data.Dataset.from_tensor_slices((x, y1, y2, y3, y4, y5))
    dataset = dataset.map(preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    
    total_samples = len(x)
    train_samples = int(total_samples * train_split)
    val_samples = int(total_samples * val_split)
    
    dataset = dataset.shuffle(buffer_size=total_samples, reshuffle_each_iteration=True)
    
    train_dataset = dataset.take(train_samples).batch(batch_size).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    val_dataset = dataset.skip(train_samples).take(val_samples).batch(batch_size).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    
    return train_dataset, val_dataset

(train_dataset, val_dataset) = tf_dataset(images, sprout, peeled, rotten, double, background, batch_size=16, 
                                           train_split=0.8, val_split=0.1)



2024-07-01 12:33:07.589510: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [17]:
# def tf_dataset(x, y1, y2, y3, y4, y5, batch_size, train_split=0.8, val_split=0.1):
#     # Combine inputs and targets into a single dataset
#     dataset = tf.data.Dataset.from_tensor_slices((x, y1, y2, y3, y4, y5))
#     dataset = dataset.map(preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    
#     # Calculate the number of samples for each split
#     total_samples = len(x)
#     train_samples = int(total_samples * train_split)
#     val_samples = int(total_samples * val_split)

#     dataset = dataset.shuffle(buffer_size=total_samples, reshuffle_each_iteration=True)
    
#     train_dataset = dataset.take(train_samples).batch(batch_size).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
#     val_dataset = dataset.skip(train_samples).take(val_samples).batch(batch_size).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    
#     # Split inputs and targets into separate datasets
#     train_images = train_dataset.map(lambda x, *y: x)
#     train_sprout = train_dataset.map(lambda x, *y: y[0])
#     train_peeled = train_dataset.map(lambda x, *y: y[1])
#     train_rotten = train_dataset.map(lambda x, *y: y[2])
#     train_double = train_dataset.map(lambda x, *y: y[3])
#     train_background = train_dataset.map(lambda x, *y: y[4])
    
#     # For validation dataset
#     val_images = val_dataset.map(lambda x, *y: x)
#     val_sprout = val_dataset.map(lambda x, *y: y[0])
#     val_peeled = val_dataset.map(lambda x, *y: y[1])
#     val_rotten = val_dataset.map(lambda x, *y: y[2])
#     val_double = val_dataset.map(lambda x, *y: y[3])
#     val_background = val_dataset.map(lambda x, *y: y[4])
    
#     # Batch the datasets
#     train_images = train_images.batch(batch_size)
#     train_sprout = train_sprout.batch(batch_size)
#     train_peeled = train_peeled.batch(batch_size)
#     train_rotten = train_rotten.batch(batch_size)
#     train_double = train_double.batch(batch_size)
#     train_background = train_background.batch(batch_size)
    
#     val_images = val_images.batch(batch_size)
#     val_sprout = val_sprout.batch(batch_size)
#     val_peeled = val_peeled.batch(batch_size)
#     val_rotten = val_rotten.batch(batch_size)
#     val_double = val_double.batch(batch_size)
#     val_background = val_background.batch(batch_size)
    
#     # Prefetch data for improved performance
#     train_images = train_images.prefetch(buffer_size=tf.data.AUTOTUNE)
#     train_sprout = train_sprout.prefetch(buffer_size=tf.data.AUTOTUNE)
#     train_peeled = train_peeled.prefetch(buffer_size=tf.data.AUTOTUNE)
#     train_rotten = train_rotten.prefetch(buffer_size=tf.data.AUTOTUNE)
#     train_double = train_double.prefetch(buffer_size=tf.data.AUTOTUNE)
#     train_background = train_background.prefetch(buffer_size=tf.data.AUTOTUNE)
    
#     val_images = val_images.prefetch(buffer_size=tf.data.AUTOTUNE)
#     val_sprout = val_sprout.prefetch(buffer_size=tf.data.AUTOTUNE)
#     val_peeled = val_peeled.prefetch(buffer_size=tf.data.AUTOTUNE)
#     val_rotten = val_rotten.prefetch(buffer_size=tf.data.AUTOTUNE)
#     val_double = val_double.prefetch(buffer_size=tf.data.AUTOTUNE)
#     val_background = val_background.prefetch(buffer_size=tf.data.AUTOTUNE)
    
#     return (train_images, train_sprout, train_peeled, train_rotten, train_double, train_background), \
#            (val_images, val_sprout, val_peeled, val_rotten, val_double, val_background)


In [18]:
# import matplotlib.pyplot as plt

# def visualize_batch(images_batch, masks_batch, batch_size=16):
#     plt.figure(figsize=(20, 20))
    
#     for i in range(batch_size):
#         img = images_batch[i]
#         mask_sprout = masks_batch[0][i][:, :, 1]
#         mask_peeled = masks_batch[1][i][:, :, 1]
#         mask_rotten = masks_batch[2][i][:, :, 1]
#         mask_double = masks_batch[3][i][:, :, 1]
#         mask_background = masks_batch[4][i][:, :, 1]
        
#         plt.subplot(batch_size, 6, i*6 + 1)
#         plt.imshow(img.astype(np.uint8))
#         plt.title('Image')
#         plt.axis('off')
        
#         plt.subplot(batch_size, 6, i*6 + 2)
#         plt.imshow(mask_sprout, cmap='gray')
#         plt.title('Sprout Mask')
#         plt.axis('off')
        
#         plt.subplot(batch_size, 6, i*6 + 3)
#         plt.imshow(mask_peeled, cmap='gray')
#         plt.title('Peeled Mask')
#         plt.axis('off')
        
#         plt.subplot(batch_size, 6, i*6 + 4)
#         plt.imshow(mask_rotten, cmap='gray')
#         plt.title('Rotten Mask')
#         plt.axis('off')
        
#         plt.subplot(batch_size, 6, i*6 + 5)
#         plt.imshow(mask_double, cmap='gray')
#         plt.title('Double Mask')
#         plt.axis('off')
        
#         plt.subplot(batch_size, 6, i*6 + 6)
#         plt.imshow(mask_background, cmap='gray')
#         plt.title('Background Mask')
#         plt.axis('off')
    
#     plt.tight_layout()
#     plt.show()




In [19]:
# # Visualize a batch after shuffling
# for batch in train_dataset.take(1):
#     images_batch, masks_batches = batch[0].numpy(), [batch[i].numpy() for i in range(1, 6)]

# visualize_batch(images_batch, masks_batches)

In [20]:
def create_dict_datasets(train_dataset, val_dataset):
    def map_fn(images, sprout, peeled, rotten, double, background):
        labels_dict = {
            "sprout": sprout,
            "peeled": peeled,
            "rotten": rotten,
            "double": double,
            "background": background
        }
        return images, labels_dict

    train_data = train_dataset.map(map_fn)
    val_data = val_dataset.map(map_fn)

    return train_data, val_data




In [21]:
train_data, val_data = create_dict_datasets(train_dataset, val_dataset)

In [22]:
# for batch in train_data.take(1):
#     images_batch, masks_dict_batch = batch[0].numpy(), {k: v.numpy() for k, v in batch[1].items()}

# visualize_batch(images_batch, masks_dict_batch)

In [23]:
#main utility functions for model
def dsc(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return score

def dice_loss(y_true, y_pred):
    loss = 1 - dsc(y_true, y_pred)
    return loss

def bce_dice_loss(y_true, y_pred):
    loss = binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)
    return loss

def iou_score(gt, pr, class_weights=1., smooth=1, per_image=True, threshold=None):
    '''
    input：
        gt: ground truth 4D keras tensor (B, H, W, C)
        pr: prediction 4D keras tensor (B, H, W, C)
        class_weights: 1. or list of class weights, len(weights) = C
        smooth: value to avoid division by zero
        per_image: if ``True``, metric is calculated as mean over images in batch (B),
            else over whole batch
        threshold: value to round predictions (use ``>`` comparison), 
        if ``None`` prediction prediction will not be round
    output：
        IoU/Jaccard score in range [0, 1]
    '''
    if per_image:
        axes = [1, 2]
    else:
        axes = [0, 1, 2]
        
    if threshold is not None:
        pr = tf.greater(pr, threshold)
        pr = tf.cast(pr, dtype=tf.float32)

    intersection = tf.reduce_sum(gt * pr, axis=axes)
    union = tf.reduce_sum(gt + pr, axis=axes) - intersection
    iou = (intersection + smooth) / (union + smooth)

    # mean per image
    if per_image:
        iou = tf.reduce_mean(iou, axis=0)

    # weighted mean per class
    iou = tf.reduce_mean(iou * class_weights)

    return iou

In [24]:
# #build model
# import segmentation_models as sm
# from keras.layers import *
# from keras import layers
# from keras.models import Model

# def SEModule(input, ratio, out_dim):
#     # bs, c, h, w
#     x = GlobalAveragePooling2D()(input)
#     excitation = Dense(units=out_dim // ratio)(x)
#     excitation = Activation('relu')(excitation)
#     excitation = Dense(units=out_dim)(excitation)
#     excitation = Activation('sigmoid')(excitation)
#     excitation = Reshape((1, 1, out_dim))(excitation)
#     scale = multiply([input, excitation])
#     return scale


# def SEUnet(nClasses, input_height=224, input_width=224):
   
#     inputs = Input(shape=(input_height, input_width, 3))
#     conv1 = Conv2D(16,
#                    3,
#                   activation='relu',
#                   padding='same',
#                   kernel_initializer='he_normal')(conv1)
#     ## quantizing 1st layer
#     quantize_annotate_layer = tfmot.quantization.keras.quantize_annotate_layer
#     conv1 = quantize_annotate_layer(conv1)
#     conv1 =  Normalization(axis=-1)(conv1)
#     inputs = Input(shape=(input_height, input_width, 3))
#     conv1 = Conv2D(16,
#                    3,
#                    activation='relu',
#                    padding='same',
#                    kernel_initializer='he_normal')(inputs)
#     conv1 = Normalization(axis=-1)(conv1)
#     # se
#     conv1 = SEModule(conv1, 4, 16)

#     pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
#     conv2 = Conv2D(32,
#                    3,
#                    activation='relu',
#                    padding='same',
#                    kernel_initializer='he_normal')(pool1)
#     conv2 = Normalization(axis=-1)(conv2)

#     conv2 = Conv2D(32,
#                    3,
#                    activation='relu',
#                    padding='same',
#                    kernel_initializer='he_normal')(conv2)
#     conv2 = Normalization(axis=-1)(conv2)

#     # se
#     conv2 = SEModule(conv2, 8, 32)

#     pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
#     conv3 = Conv2D(64,
#                    3,
#                    activation='relu',
#                    padding='same',
#                    kernel_initializer='he_normal')(pool2)
#     conv3 = Normalization(axis=-1)(conv3)

#     conv3 = Conv2D(64,
#                    3,
#                    activation='relu',
#                    padding='same',
#                    kernel_initializer='he_normal')(conv3)
#     conv3 = Normalization(axis=-1)(conv3)

#     # se
#     conv3 = SEModule(conv3, 8, 64)

#     pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
#     conv4 = Conv2D(128,
#                    3,
#                    activation='relu',
#                    padding='same',
#                    kernel_initializer='he_normal')(pool3)
#     conv4 = Normalization(axis=-1)(conv4)

#     conv4 = Conv2D(128,
#                    3,
#                    activation='relu',
#                    padding='same',
#                    kernel_initializer='he_normal')(conv4)
#     conv4 = Normalization(axis=-1)(conv4)

#     # se
#     conv4 = SEModule(conv4, 16, 128)

#     pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

#     conv5 = Conv2D(256,
#                    3,
#                    activation='relu',
#                    padding='same',
#                    kernel_initializer='he_normal')(pool4)
#     conv5 = Normalization(axis=-1)(conv5)
#     conv5 = Conv2D(256,
#                    3,
#                    activation='relu',
#                    padding='same',
#                    kernel_initializer='he_normal')(conv5)
#     conv5 = BatchNormalization()(conv5)

#     # se
#     conv5 = SEModule(conv5, 16, 256)

#     up6 = Conv2D(128,
#                  2,
#                  activation='relu',
#                  padding='same',
#                  kernel_initializer='he_normal')(UpSampling2D(size=(2,
#                                                                     2))(conv5))
#     up6 = Normalization(axis=-1)(up6)

#     merge6 = concatenate([conv4, up6], axis=3)
#     conv6 = Conv2D(128,
#                    3,
#                    activation='relu',
#                    padding='same',
#                    kernel_initializer='he_normal')(merge6)
#     conv6 = Normalization(axis=-1)(conv6)

#     conv6 = Conv2D(128,
#                    3,
#                    activation='relu',
#                    padding='same',
#                    kernel_initializer='he_normal')(conv6)
#     conv6 = BatchNormalization()(conv6)

#     # se
#     conv6 = SEModule(conv6, 16, 128)

#     up7 = Conv2D(64,
#                  2,
#                  activation='relu',
#                  padding='same',
#                  kernel_initializer='he_normal')(UpSampling2D(size=(2,
#                                                                     2))(conv6))
#     up7 = Normalization(axis=-1)(up7)

#     merge7 = concatenate([conv3, up7], axis=3)
#     conv7 = Conv2D(64,
#                    3,
#                    activation='relu',
#                    padding='same',
#                    kernel_initializer='he_normal')(merge7)
#     conv7 = Normalization(axis=-1)(conv7)

#     conv7 = Conv2D(64,
#                    3,
#                    activation='relu',
#                    padding='same',
#                    kernel_initializer='he_normal')(conv7)
#     conv7 = Normalization(axis=-1)(conv7)

#     # se
#     conv7 = SEModule(conv7, 8, 64)

#     up8 = Conv2D(32,
#                  2,
#                  activation='relu',
#                  padding='same',
#                  kernel_initializer='he_normal')(UpSampling2D(size=(2,
#                                                                     2))(conv7))
#     up8 = Normalization(axis=-1)(up8)

#     merge8 = concatenate([conv2, up8], axis=3)
#     conv8 = Conv2D(32,
#                    3,
#                    activation='relu',
#                    padding='same',
#                    kernel_initializer='he_normal')(merge8)
#     conv8 = Normalization(axis=-1)(conv8)

#     conv8 = Conv2D(32,
#                    3,
#                    activation='relu',
#                    padding='same',
#                    kernel_initializer='he_normal')(conv8)
#     conv8 =Normalization(axis=-1)(conv8)

#     # se
#     conv8 = SEModule(conv8, 4, 32)

#     up9 = Conv2D(16,
#                  2,
#                  activation='relu',
#                  padding='same',
#                  kernel_initializer='he_normal')(UpSampling2D(size=(2,
#                                                                     2))(conv8))
#     up9 = Normalization(axis=-1)(up9)

#     merge9 = concatenate([conv1, up9], axis=3)
#     conv9 = Conv2D(16,
#                    3,
#                    activation='relu',
#                    padding='same',
#                    kernel_initializer='he_normal')(merge9)
#     conv9 = Normalization(axis=-1)(conv9)

#     conv9 = Conv2D(16,
#                    3,
#                    activation='relu',
#                    padding='same',
#                    kernel_initializer='he_normal')(conv9)
#     conv9 = Normalization(axis=-1)(conv9)

#     # se
#     conv9 = SEModule(conv9, 2, 16)

#     conv10 = Conv2D(nClasses, (3, 3), padding='same')(conv9)
#     conv10 = Normalization(axis=-1)(conv10)

#     outputHeight = Model(inputs, conv10).output_shape[1]
#     outputWidth = Model(inputs, conv10).output_shape[2]

#     out = (Reshape((outputHeight * outputWidth, nClasses)))(conv10)
#     out = Activation('softmax')(out)

#     model = Model(inputs,out)
#     model.outputHeight = outputHeight
#     model.outputWidth = outputWidth

#     return model

# model1 = SEUnet(nClasses=5)
# model1.summary()

In [25]:
# # import segmentation_models as sm
# from keras.layers import *
# from keras import layers
# from keras.models import Model
# import tensorflow_model_optimization as tfmot

# def SEModule(input, ratio, out_dim):
#     x = GlobalAveragePooling2D()(input)
#     excitation = Dense(units=out_dim // ratio)(x)
#     excitation = Activation('relu')(excitation)
#     excitation = Dense(units=out_dim)(excitation)
#     excitation = Activation('sigmoid')(excitation)
#     excitation = Reshape((1, 1, out_dim))(excitation)
#     scale = multiply([input, excitation])
#     return scale

# def SEUnet(nClasses, input_height=224, input_width=224):
#     inputs = Input(shape=(input_height, input_width, 3))
    
#     quantize_annotate_layer = tfmot.quantization.keras.quantize_annotate_layer

#     conv1 = quantize_annotate_layer(
#         Conv2D(16, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(inputs)
    
#     conv1 = Normalization(axis=-1)(conv1)
    
#     # SE
#     conv1 = SEModule(conv1, 4, 16)

#     pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
#     conv2 = quantize_annotate_layer(
#         Conv2D(32, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(pool1)
#     conv2 = Normalization(axis=-1)(conv2)
#     conv2 = quantize_annotate_layer(
#         Conv2D(32, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(conv2)
#     conv2 = Normalization(axis=-1)(conv2)
    
#     # SE
#     conv2 = SEModule(conv2, 8, 32)

#     pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
#     conv3 = quantize_annotate_layer(
#         Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(pool2)
#     conv3 = Normalization(axis=-1)(conv3)
#     conv3 = quantize_annotate_layer(
#         Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(conv3)
#     conv3 = Normalization(axis=-1)(conv3)
    
#     # SE
#     conv3 = SEModule(conv3, 8, 64)

#     pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
#     conv4 = quantize_annotate_layer(
#         Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(pool3)
#     conv4 = Normalization(axis=-1)(conv4)
#     conv4 = quantize_annotate_layer(
#         Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(conv4)
#     conv4 = Normalization(axis=-1)(conv4)
    
#     # SE
#     conv4 = SEModule(conv4, 16, 128)

#     pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
#     conv5 = quantize_annotate_layer(
#         Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(pool4)
#     conv5 = Normalization(axis=-1)(conv5)
#     conv5 = quantize_annotate_layer(
#         Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(conv5)
#     conv5 = BatchNormalization()(conv5)
    
#     # SE
#     conv5 = SEModule(conv5, 16, 256)

#     up6 = quantize_annotate_layer(
#         Conv2D(128, 2, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(UpSampling2D(size=(2, 2))(conv5))
#     up6 = Normalization(axis=-1)(up6)
#     merge6 = concatenate([conv4, up6], axis=3)
#     conv6 = quantize_annotate_layer(
#         Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(merge6)
#     conv6 = Normalization(axis=-1)(conv6)
#     conv6 = quantize_annotate_layer(
#         Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(conv6)
#     conv6 = BatchNormalization()(conv6)
    
#     # SE
#     conv6 = SEModule(conv6, 16, 128)

#     up7 = quantize_annotate_layer(
#         Conv2D(64, 2, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(UpSampling2D(size=(2, 2))(conv6))
#     up7 = Normalization(axis=-1)(up7)
#     merge7 = concatenate([conv3, up7], axis=3)
#     conv7 = quantize_annotate_layer(
#         Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(merge7)
#     conv7 = Normalization(axis=-1)(conv7)
#     conv7 = quantize_annotate_layer(
#         Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(conv7)
#     conv7 = Normalization(axis=-1)(conv7)
    
#     # SE
#     conv7 = SEModule(conv7, 8, 64)

#     up8 = quantize_annotate_layer(
#         Conv2D(32, 2, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(UpSampling2D(size=(2, 2))(conv7))
#     up8 = Normalization(axis=-1)(up8)
#     merge8 = concatenate([conv2, up8], axis=3)
#     conv8 = quantize_annotate_layer(
#         Conv2D(32, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(merge8)
#     conv8 = Normalization(axis=-1)(conv8)
#     conv8 = quantize_annotate_layer(
#         Conv2D(32, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(conv8)
#     conv8 = Normalization(axis=-1)(conv8)
    
#     # SE
#     conv8 = SEModule(conv8, 4, 32)

#     up9 = quantize_annotate_layer(
#         Conv2D(16, 2, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(UpSampling2D(size=(2, 2))(conv8))
#     up9 = Normalization(axis=-1)(up9)
#     merge9 = concatenate([conv1, up9], axis=3)
#     conv9 = quantize_annotate_layer(
#         Conv2D(16, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(merge9)
#     conv9 = Normalization(axis=-1)(conv9)
#     conv9 = quantize_annotate_layer(
#         Conv2D(16, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(conv9)
#     conv9 = Normalization(axis=-1)(conv9)
    
#     # SE
#     conv9 = SEModule(conv9, 2, 16)

#     conv10 = quantize_annotate_layer(
#         Conv2D(nClasses, (3, 3), padding='same')
#     )(conv9)
#     conv10 = Normalization(axis=-1)(conv10)

#     outputHeight = Model(inputs, conv10).output_shape[1]
#     outputWidth = Model(inputs, conv10).output_shape[2]

#     out = Reshape((outputHeight * outputWidth, nClasses))(conv10)
#     out = Activation('softmax')(out)

#     model = Model(inputs, out)
#     model.outputHeight = outputHeight
#     model.outputWidth = outputWidth

#     return model

# model1 = SEUnet(nClasses=5)
# model1.summary()


## QAT without quantconfig

In [26]:
# import segmentation_models as sm
# from keras.layers import *
# from keras import layers
# from keras.models import Model
# import tensorflow_model_optimization as tfmot

# def SEModule(input, ratio, out_dim):
#     x = GlobalAveragePooling2D()(input)
#     excitation = Dense(units=out_dim // ratio)(x)
#     excitation = Activation('relu')(excitation)
#     excitation = Dense(units=out_dim)(excitation)
#     excitation = Activation('sigmoid')(excitation)
#     excitation = Reshape((1, 1, out_dim))(excitation)
#     scale = multiply([input, excitation])
#     return scale

# def SEUnet(nClasses, input_height=224, input_width=224):
#     inputs = Input(shape=(input_height, input_width, 3))
    
#     quantize_annotate_layer = tfmot.quantization.keras.quantize_annotate_layer

#     conv1 = quantize_annotate_layer(
#         Conv2D(16, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(inputs)
    
#     conv1 = BatchNormalization()(conv1)
#     conv1 = quantize_annotate_layer(
#         Conv2D(16, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(inputs)
#     conv1 = BatchNormalization()(conv1)
#     # SE
#     conv1 = SEModule(conv1, 4, 16)

#     pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
#     conv2 = quantize_annotate_layer(
#         Conv2D(32, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(pool1)
#     conv2 = BatchNormalization()(conv2)
#     conv2 = quantize_annotate_layer(
#         Conv2D(32, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(conv2)
#     conv2 = BatchNormalization()(conv2)
    
#     # SE
#     conv2 = SEModule(conv2, 8, 32)

#     pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
#     conv3 = quantize_annotate_layer(
#         Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(pool2)
#     conv3 = BatchNormalization()(conv3)
#     conv3 = quantize_annotate_layer(
#         Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(conv3)
#     conv3 = BatchNormalization()(conv3)
    
#     # SE
#     conv3 = SEModule(conv3, 8, 64)

#     pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
#     conv4 = quantize_annotate_layer(
#         Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(pool3)
#     conv4 = BatchNormalization()(conv4)
#     conv4 = quantize_annotate_layer(
#         Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(conv4)
#     conv4 = BatchNormalization()(conv4)
    
#     # SE
#     conv4 = SEModule(conv4, 16, 128)

#     pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
#     conv5 = quantize_annotate_layer(
#         Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(pool4)
#     conv5 = BatchNormalization()(conv5)
#     conv5 = quantize_annotate_layer(
#         Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(conv5)
#     conv5 = BatchNormalization()(conv5)
    
#     # SE
#     conv5 = SEModule(conv5, 16, 256)

#     up6 = quantize_annotate_layer(
#         Conv2D(128, 2, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(UpSampling2D(size=(2, 2))(conv5))
#     up6 = BatchNormalization()(up6)
#     merge6 = concatenate([conv4, up6], axis=3)
#     conv6 = quantize_annotate_layer(
#         Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(merge6)
#     conv6 = BatchNormalization()(conv6)
#     conv6 = quantize_annotate_layer(
#         Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(conv6)
#     conv6 = BatchNormalization()(conv6)
    
#     # SE
#     conv6 = SEModule(conv6, 16, 128)

#     up7 = quantize_annotate_layer(
#         Conv2D(64, 2, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(UpSampling2D(size=(2, 2))(conv6))
#     up7 = BatchNormalization()(up7)
#     merge7 = concatenate([conv3, up7], axis=3)
#     conv7 = quantize_annotate_layer(
#         Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(merge7)
#     conv7 = BatchNormalization()(conv7)
#     conv7 = quantize_annotate_layer(
#         Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(conv7)
#     conv7 = BatchNormalization()(conv7)
    
#     # SE
#     conv7 = SEModule(conv7, 8, 64)

#     up8 = quantize_annotate_layer(
#         Conv2D(32, 2, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(UpSampling2D(size=(2, 2))(conv7))
#     up8 = BatchNormalization()(up8)
#     merge8 = concatenate([conv2, up8], axis=3)
#     conv8 = quantize_annotate_layer(
#         Conv2D(32, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(merge8)
#     conv8 = BatchNormalization()(conv8)
#     conv8 = quantize_annotate_layer(
#         Conv2D(32, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(conv8)
#     conv8 = BatchNormalization()(conv8)
    
#     # SE
#     conv8 = SEModule(conv8, 4, 32)

#     up9 = quantize_annotate_layer(
#         Conv2D(16, 2, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(UpSampling2D(size=(2, 2))(conv8))
#     up9 = BatchNormalization()(up9)
#     merge9 = concatenate([conv1, up9], axis=3)
#     conv9 = quantize_annotate_layer(
#         Conv2D(16, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(merge9)
#     conv9 = BatchNormalization()(conv9)
#     conv9 = quantize_annotate_layer(
#         Conv2D(16, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(conv9)
#     conv9 = BatchNormalization()(conv9)
    
#     # SE
#     conv9 = SEModule(conv9, 2, 16)

#     conv10 = quantize_annotate_layer(
#         Conv2D(nClasses, (3, 3), padding='same')
#     )(conv9)
#     conv10 = BatchNormalization()(conv10)

#     outputHeight = Model(inputs, conv10).output_shape[1]
#     outputWidth = Model(inputs, conv10).output_shape[2]

#     out = Reshape((outputHeight * outputWidth, nClasses))(conv10)
#     out = Activation('softmax')(out)

#     model = Model(inputs, out)
#     model.outputHeight = outputHeight
#     model.outputWidth = outputWidth

#     return model

# model1 = SEUnet(nClasses=5)
# model1.summary()


## QAT with Quant config

In [27]:
# class ConvQuantizeConfig(tfmot.quantization.keras.QuantizeConfig):
#     def get_weights_and_quantizers(self, layer):
#         return [(layer.kernel, tfmot.quantization.keras.quantizers.LastValueQuantizer(num_bits=8, symmetric=True, narrow_range=False, per_axis=False))]

#     def get_activations_and_quantizers(self, layer):
#         return [(layer.activation, tfmot.quantization.keras.quantizers.MovingAverageQuantizer(num_bits=8, symmetric=False, narrow_range=False))]

#     def set_quantize_weights(self, layer, quantize_weights):
#         layer.kernel = quantize_weights[0]

#     def set_quantize_activations(self, layer, quantize_activations):
#         pass  # No need to explicitly set quantized activations

#     def get_output_quantizers(self, layer):
#         return [tfmot.quantization.keras.quantizers.MovingAverageQuantizer(num_bits=8, symmetric=False, narrow_range=False)]

#     def get_config(self):
#         return {}


In [28]:
import tensorflow_model_optimization as tfmot
from tensorflow_model_optimization.python.core.quantization.keras.quantizers import MovingAverageQuantizer, LastValueQuantizer
from tensorflow_model_optimization.python.core.quantization.keras.quantize_config import QuantizeConfig
from keras.layers import Layer

# Custom QuantizeConfig for Conv2D layers
class ConvQuantizeConfig(QuantizeConfig):
    def get_weights_and_quantizers(self, layer):
        return [(layer.kernel, LastValueQuantizer(num_bits=4, per_axis=False, symmetric=False, narrow_range=False))]

    def get_activations_and_quantizers(self, layer):
        return [(layer.activation, MovingAverageQuantizer(num_bits=4, per_axis=False, symmetric=False, narrow_range=False))]

    def set_quantize_weights(self, layer, quantize_weights):
        layer.kernel = quantize_weights[0]

    def set_quantize_activations(self, layer, quantize_activations):
        layer.activation = quantize_activations[0]

    def get_output_quantizers(self, layer):
        # Does not quantize output, since we return an empty list.
        return []

    def get_config(self):
        return {}


In [29]:
tf.keras.utils.register_keras_serializable(package='QuantizeConfig')(ConvQuantizeConfig)

__main__.ConvQuantizeConfig

In [30]:
#quantize_annotate_layer = tfmot.quantization.keras.quantize_annotate_layer

import segmentation_models as sm
from keras.layers import *
from keras import layers
from keras.models import Model


def SEModule(input, ratio, out_dim):
    x = GlobalAveragePooling2D()(input)
    excitation = Dense(units=out_dim // ratio)(x)
    excitation = Activation('relu')(excitation)
    excitation = Dense(units=out_dim)(excitation)
    excitation = Activation('sigmoid')(excitation)
    excitation = Reshape((1, 1, out_dim))(excitation)
    scale = multiply([input, excitation])
    return scale

def SEUnet(nClasses, input_height=224, input_width=224):
    inputs = Input(shape=(input_height, input_width, 3))
    quantize_annotate_layer = tfmot.quantization.keras.quantize_annotate_layer
    
    conv1 = quantize_annotate_layer(
        Conv2D(16, 3, activation='relu', padding='same', kernel_initializer='he_normal'),
        quantize_config=ConvQuantizeConfig()
    )(inputs)
    
    conv1 = BatchNormalization()(conv1)
    conv1 = quantize_annotate_layer(
        Conv2D(16, 3, activation='relu', padding='same', kernel_initializer='he_normal'),
        quantize_config=ConvQuantizeConfig()
    )(conv1)
    conv1 = BatchNormalization()(conv1)
    # SE
    conv1 = SEModule(conv1, 4, 16)

    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = quantize_annotate_layer(
        Conv2D(32, 3, activation='relu', padding='same', kernel_initializer='he_normal'),
        quantize_config=ConvQuantizeConfig()
    )(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = quantize_annotate_layer(
        Conv2D(32, 3, activation='relu', padding='same', kernel_initializer='he_normal'),
        quantize_config=ConvQuantizeConfig()
    )(conv2)
    conv2 = BatchNormalization()(conv2)
    
    # SE
    conv2 = SEModule(conv2, 8, 32)

    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = quantize_annotate_layer(
        Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal'),
        quantize_config=ConvQuantizeConfig()
    )(pool2)
    conv3 = BatchNormalization()(conv3)
    conv3 = quantize_annotate_layer(
        Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal'),
        quantize_config=ConvQuantizeConfig()
    )(conv3)
    conv3 = BatchNormalization()(conv3)
    
    # SE
    conv3 = SEModule(conv3, 8, 64)

    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = quantize_annotate_layer(
        Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal'),
        quantize_config=ConvQuantizeConfig()
    )(pool3)
    conv4 = BatchNormalization()(conv4)
    conv4 = quantize_annotate_layer(
        Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal'),
        quantize_config=ConvQuantizeConfig()
    )(conv4)
    conv4 = BatchNormalization()(conv4)
    
    # SE
    conv4 = SEModule(conv4, 16, 128)

    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    conv5 = quantize_annotate_layer(
        Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal'),
        quantize_config=ConvQuantizeConfig()
    )(pool4)
    conv5 = BatchNormalization()(conv5)
    conv5 = quantize_annotate_layer(
        Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal'),
        quantize_config=ConvQuantizeConfig()
    )(conv5)
    conv5 = BatchNormalization()(conv5)
    
    # SE
    conv5 = SEModule(conv5, 16, 256)

    up6 = quantize_annotate_layer(
        Conv2D(128, 2, activation='relu', padding='same', kernel_initializer='he_normal'),
        quantize_config=ConvQuantizeConfig()
    )(UpSampling2D(size=(2, 2))(conv5))
    up6 = BatchNormalization()(up6)
    merge6 = concatenate([conv4, up6], axis=3)
    conv6 = quantize_annotate_layer(
        Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal'),
        quantize_config=ConvQuantizeConfig()
    )(merge6)
    conv6 = BatchNormalization()(conv6)
    conv6 = quantize_annotate_layer(
        Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal'),
        quantize_config=ConvQuantizeConfig()
    )(conv6)
    conv6 = BatchNormalization()(conv6)
    
    # SE
    conv6 = SEModule(conv6, 16, 128)

    up7 = quantize_annotate_layer(
        Conv2D(64, 2, activation='relu', padding='same', kernel_initializer='he_normal'),
        quantize_config=ConvQuantizeConfig()
    )(UpSampling2D(size=(2, 2))(conv6))
    up7 = BatchNormalization()(up7)
    merge7 = concatenate([conv3, up7], axis=3)
    conv7 = quantize_annotate_layer(
        Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal'),
        quantize_config=ConvQuantizeConfig()
    )(merge7)
    conv7 = BatchNormalization()(conv7)
    conv7 = quantize_annotate_layer(
        Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal'),
        quantize_config=ConvQuantizeConfig()
    )(conv7)
    conv7 = BatchNormalization()(conv7)
    
    # SE
    conv7 = SEModule(conv7, 8, 64)

    up8 = quantize_annotate_layer(
        Conv2D(32, 2, activation='relu', padding='same', kernel_initializer='he_normal'),
        quantize_config=ConvQuantizeConfig()
    )(UpSampling2D(size=(2, 2))(conv7))
    up8 = BatchNormalization()(up8)
    merge8 = concatenate([conv2, up8], axis=3)
    conv8 = quantize_annotate_layer(
        Conv2D(32, 3, activation='relu', padding='same', kernel_initializer='he_normal'),
        quantize_config=ConvQuantizeConfig()
    )(merge8)
    conv8 = BatchNormalization()(conv8)
    conv8 = quantize_annotate_layer(
        Conv2D(32, 3, activation='relu', padding='same', kernel_initializer='he_normal'),
        quantize_config=ConvQuantizeConfig()
    )(conv8)
    conv8 = BatchNormalization()(conv8)
    
    # SE
    conv8 = SEModule(conv8, 4, 32)

    up9 = quantize_annotate_layer(
        Conv2D(16, 2, activation='relu', padding='same', kernel_initializer='he_normal'),
        quantize_config=ConvQuantizeConfig()
    )(UpSampling2D(size=(2, 2))(conv8))
    up9 = BatchNormalization()(up9)
    merge9 = concatenate([conv1, up9], axis=3)
    conv9 = quantize_annotate_layer(
        Conv2D(16, 3, activation='relu', padding='same', kernel_initializer='he_normal'),
        quantize_config=ConvQuantizeConfig()
    )(merge9)
    conv9 = BatchNormalization()(conv9)
    conv9 = quantize_annotate_layer(
        Conv2D(16, 3, activation='relu', padding='same', kernel_initializer='he_normal'),
        quantize_config=ConvQuantizeConfig()
    )(conv9)
    conv9 = BatchNormalization()(conv9)
    
    # SE
    conv9 = SEModule(conv9, 2, 16)

    conv10 = quantize_annotate_layer(
        Conv2D(nClasses, (3, 3), padding='same'),
        quantize_config=ConvQuantizeConfig()
    )(conv9)
    conv10 = BatchNormalization()(conv10)

    outputHeight = Model(inputs, conv10).output_shape[1]
    outputWidth = Model(inputs, conv10).output_shape[2]

    out = Reshape((outputHeight * outputWidth, nClasses))(conv10)
    out = Activation('softmax')(out)

    model = Model(inputs, out)
    model.outputHeight = outputHeight
    model.outputWidth = outputWidth

    return model
model1 = SEUnet(nClasses=5)
model1.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 quantize_annotate (QuantizeAnn  (None, 224, 224, 16  448        ['input_1[0][0]']                
 otate)                         )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 224, 224, 16  64         ['quantize_annotate[0][0]']      
 alization)                     )                                                           

In [31]:
x = model1.get_layer(index=-3).output

# folder=["Sprout","Black_smut","Rotten","Background"]
out0 = Conv2D(2, (1, 1), activation='softmax',name='sprout')(x)
out1 = Conv2D(2, (1, 1), activation='softmax',name='peeled')(x)
out2 = Conv2D(2, (1, 1), activation='softmax',name='rotten')(x)
out3 = Conv2D(2, (1, 1), activation='softmax',name='double')(x)
out4 = Conv2D(2, (1, 1), activation='softmax',name='background')(x)
model_new = Model(inputs = model1.input,outputs = [out0,out1,out2,out3,out4])
# Register custom objects and apply quantization
# with tfmot.quantization.keras.quantize_scope({'ConvQuantizeConfig': ConvQuantizeConfig}):
#     quantized_model = tfmot.quantization.keras.quantize_apply(model_new)

model_new.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 quantize_annotate (QuantizeAnn  (None, 224, 224, 16  448        ['input_1[0][0]']                
 otate)                         )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 224, 224, 16  64         ['quantize_annotate[0][0]']      
 alization)                     )                                                           

In [32]:
quantized_model = tfmot.quantization.keras.quantize_apply(model_new)

In [33]:
from segmentation_models.metrics import  f1_score,precision,recall
from segmentation_models.metrics import  f1_score
from tensorflow.keras.callbacks import TensorBoard
import datetime

In [34]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [35]:

from tensorflow.keras.optimizers import Adam
model_name = 'onin_28th_june_spplrobg'
date = '28thjune2024'
learning_rate = 0.001
# num_epochs = 20




# Initialize the optimizer
opt = Adam(lr=learning_rate)



# Define losses, metrics, and loss weights
losses = {"sprout": sm.losses.bce_jaccard_loss, "peeled": sm.losses.bce_jaccard_loss, "rotten": sm.losses.bce_jaccard_loss,"double": sm.losses.bce_jaccard_loss,"background": sm.losses.bce_jaccard_loss}
metrics = {"sprout": [iou_score, f1_score,precision,recall], "peeled": [iou_score, f1_score,precision,recall], "rotten": [iou_score, f1_score,precision,recall],"double": [iou_score, f1_score,precision,recall], "background": [iou_score, f1_score,precision,recall]}
loss_weights = {"sprout": 1.0, "peeled": 1.0, "rotten": 1.0, "double": 1.0, "background": 2.0}

print("[INFO] compiling model...")

# Define callbacks
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

LR_Reduce_callback = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=10, verbose=1, mode='auto')
#model_ckpt1 = ModelCheckpoint(model_name+'_'+date+'.h5', monitor='loss', save_weights_only=False,save_best_only=True, period=1)
model_ckpt2 = ModelCheckpoint(model_name+'_'+date+'_weights.h5', monitor='loss', save_weights_only=True,save_best_only=True, period=1)
# reduce_rl_plateau = CustomReduceLRoP(patience=4, 
#                               verbose=1, 
#                               optim_lr=opt.learning_rate, 
#                               mode='auto',
#                               factor=0.1)
# Compile the model using the initialized optimizer
quantized_model.compile(optimizer=opt, loss=losses, metrics=metrics)

[INFO] compiling model...


/home/agrograde/.local/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [36]:

H = quantized_model.fit(train_data,
                  epochs=1,
                  validation_data=val_data,
                  callbacks=[tensorboard_callback, LR_Reduce_callback, model_ckpt2],
                  batch_size=16)

2024-07-01 12:34:18.793110: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 13 of 30
2024-07-01 12:34:24.644195: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:415] Shuffle buffer filled.


2/2 [==============================] - ETA: 0s - loss: 7.8338 - sprout_loss: 1.5790 - peeled_loss: 1.5481 - rotten_loss: 1.5799 - double_loss: 1.6105 - background_loss: 1.5164 - sprout_iou_score: 0.2514 - sprout_f1-score: 0.3395 - sprout_precision: 0.5002 - sprout_recall: 0.5067 - peeled_iou_score: 0.2603 - peeled_f1-score: 0.3591 - peeled_precision: 0.5006 - peeled_recall: 0.5073 - rotten_iou_score: 0.2559 - rotten_f1-score: 0.3496 - rotten_precision: 0.5002 - rotten_recall: 0.5067 - double_iou_score: 0.2508 - double_f1-score: 0.3340 - double_precision: 0.5000 - double_recall: 0.7508 - background_iou_score: 0.3273 - background_f1-score: 0.4762 - background_precision: 0.5267 - background_recall: 0.5460 

2024-07-01 12:34:45.172987: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 13 of 30
2024-07-01 12:34:50.561188: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 28 of 30
2024-07-01 12:34:50.561255: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:415] Shuffle buffer filled.


2/2 [==============================] - 52s 24s/step - loss: 7.8338 - sprout_loss: 1.5790 - peeled_loss: 1.5481 - rotten_loss: 1.5799 - double_loss: 1.6105 - background_loss: 1.5164 - sprout_iou_score: 0.2514 - sprout_f1-score: 0.3395 - sprout_precision: 0.5002 - sprout_recall: 0.5067 - peeled_iou_score: 0.2603 - peeled_f1-score: 0.3591 - peeled_precision: 0.5006 - peeled_recall: 0.5073 - rotten_iou_score: 0.2559 - rotten_f1-score: 0.3496 - rotten_precision: 0.5002 - rotten_recall: 0.5067 - double_iou_score: 0.2508 - double_f1-score: 0.3340 - double_precision: 0.5000 - double_recall: 0.7508 - background_iou_score: 0.3273 - background_f1-score: 0.4762 - background_precision: 0.5267 - background_recall: 0.5460 - val_loss: 6.9533 - val_sprout_loss: 1.3444 - val_peeled_loss: 1.4795 - val_rotten_loss: 1.3161 - val_double_loss: 1.4794 - val_background_loss: 1.3338 - val_sprout_iou_score: 0.2887 - val_sprout_f1-score: 0.3673 - val_sprout_precision: 0.4998 - val_sprout_recall: 0.4800 - val_peel

In [37]:
# import tensorflow as tf

# # Function to represent the dataset for calibration
# def representative_data_gen():
#     for input_value in tf.data.Dataset.from_tensor_slices(train_data).batch(1).take(10):
#         yield [input_value]

# # Convert the model to a TFLite model
# converter = tf.lite.TFLiteConverter.from_keras_model(quantized_model)
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# converter.representative_dataset = representative_data_gen
# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# converter.inference_input_type = tf.uint8  # or tf.int8
# converter.inference_output_type = tf.uint8  # or tf.int8

# tflite_model = converter.convert()

# # Save the model
# with open('quantized_model.tflite2', 'wb') as f:
#     f.write(tflite_model)


In [38]:
converter = tf.lite.TFLiteConverter.from_keras_model(quantized_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
float_tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpu41kl8ri/assets


INFO:tensorflow:Assets written to: /tmp/tmpu41kl8ri/assets
/home/agrograde/.local/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2024-07-01 12:35:15.800715: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2024-07-01 12:35:15.800750: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2024-07-01 12:35:15.801301: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpu41kl8ri
2024-07-01 12:35:15.831807: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2024-07-01 12:35:15.831856: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpu41kl8ri
2024-07-01 12:35:15.912183: I tensorflow/compiler/mlir/mlir_gra

In [39]:
with open("tflite_model.tflite","wb") as f:
    f.write(float_tflite_model)

In [ ]:
# import tensorflow_model_optimization as tfmot``````````

In [ ]:
# quntize_annotate_layer = tfmot.quantization.keras.quantize_annotate_layer

In [ ]:
# #build model
# import segmentation_models as sm
# from keras.layers import *
# from keras import layers
# from keras.models import Model
# import tensorflow_model_optimization as tfmot

# def SEModule(input, ratio, out_dim):
#     x = GlobalAveragePooling2D()(input)
#     excitation = Dense(units=out_dim // ratio)(x)
#     excitation = Activation('relu')(excitation)
#     excitation = Dense(units=out_dim)(excitation)
#     excitation = Activation('sigmoid')(excitation)
#     excitation = Reshape((1, 1, out_dim))(excitation)
#     scale = multiply([input, excitation])
#     return scale

# def SEUnet(nClasses, input_height=224, input_width=224):
#     inputs = Input(shape=(input_height, input_width, 3))
    
#     quantize_annotate_layer = tfmot.quantization.keras.quantize_annotate_layer

#     conv1 = quantize_annotate_layer(
#         Conv2D(16, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(inputs)
    
#     conv1 = BatchNormalization()(conv1)
#     conv1 = quantize_annotate_layer(
#         Conv2D(16, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(inputs)
#     conv1 = BatchNormalization()(conv1)
#     # SE
#     conv1 = SEModule(conv1, 4, 16)

#     pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
#     conv2 = quantize_annotate_layer(
#         Conv2D(32, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(pool1)
#     conv2 = BatchNormalization()(conv2)
#     conv2 = quantize_annotate_layer(
#         Conv2D(32, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(conv2)
#     conv2 = BatchNormalization()(conv2)
    
#     # SE
#     conv2 = SEModule(conv2, 8, 32)

#     pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
#     conv3 = quantize_annotate_layer(
#         Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(pool2)
#     conv3 = BatchNormalization()(conv3)
#     conv3 = quantize_annotate_layer(
#         Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(conv3)
#     conv3 = BatchNormalization()(conv3)
    
#     # SE
#     conv3 = SEModule(conv3, 8, 64)

#     pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
#     conv4 = quantize_annotate_layer(
#         Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(pool3)
#     conv4 = BatchNormalization()(conv4)
#     conv4 = quantize_annotate_layer(
#         Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(conv4)
#     conv4 = BatchNormalization()(conv4)
    
#     # SE
#     conv4 = SEModule(conv4, 16, 128)

#     pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
#     conv5 = quantize_annotate_layer(
#         Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(pool4)
#     conv5 = BatchNormalization()(conv5)
#     conv5 = quantize_annotate_layer(
#         Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(conv5)
#     conv5 = BatchNormalization()(conv5)
    
#     # SE
#     conv5 = SEModule(conv5, 16, 256)

#     up6 = quantize_annotate_layer(
#         Conv2D(128, 2, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(UpSampling2D(size=(2, 2))(conv5))
#     up6 = BatchNormalization()(up6)
#     merge6 = concatenate([conv4, up6], axis=3)
#     conv6 = quantize_annotate_layer(
#         Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(merge6)
#     conv6 = BatchNormalization()(conv6)
#     conv6 = quantize_annotate_layer(
#         Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(conv6)
#     conv6 = BatchNormalization()(conv6)
    
#     # SE
#     conv6 = SEModule(conv6, 16, 128)

#     up7 = quantize_annotate_layer(
#         Conv2D(64, 2, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(UpSampling2D(size=(2, 2))(conv6))
#     up7 = BatchNormalization()(up7)
#     merge7 = concatenate([conv3, up7], axis=3)
#     conv7 = quantize_annotate_layer(
#         Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(merge7)
#     conv7 = BatchNormalization()(conv7)
#     conv7 = quantize_annotate_layer(
#         Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(conv7)
#     conv7 = BatchNormalization()(conv7)
    
#     # SE
#     conv7 = SEModule(conv7, 8, 64)

#     up8 = quantize_annotate_layer(
#         Conv2D(32, 2, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(UpSampling2D(size=(2, 2))(conv7))
#     up8 = BatchNormalization()(up8)
#     merge8 = concatenate([conv2, up8], axis=3)
#     conv8 = quantize_annotate_layer(
#         Conv2D(32, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(merge8)
#     conv8 = BatchNormalization()(conv8)
#     conv8 = quantize_annotate_layer(
#         Conv2D(32, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(conv8)
#     conv8 = BatchNormalization()(conv8)
    
#     # SE
#     conv8 = SEModule(conv8, 4, 32)

#     up9 = quantize_annotate_layer(
#         Conv2D(16, 2, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(UpSampling2D(size=(2, 2))(conv8))
#     up9 = BatchNormalization()(up9)
#     merge9 = concatenate([conv1, up9], axis=3)
#     conv9 = quantize_annotate_layer(
#         Conv2D(16, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(merge9)
#     conv9 = BatchNormalization()(conv9)
#     conv9 = quantize_annotate_layer(
#         Conv2D(16, 3, activation='relu', padding='same', kernel_initializer='he_normal')
#     )(conv9)
#     conv9 = BatchNormalization()(conv9)
    
#     # SE
#     conv9 = SEModule(conv9, 2, 16)

#     conv10 = quantize_annotate_layer(
#         Conv2D(nClasses, (3, 3), padding='same')
#     )(conv9)
#     conv10 = BatchNormalization()(conv10)

#     outputHeight = Model(inputs, conv10).output_shape[1]
#     outputWidth = Model(inputs, conv10).output_shape[2]

#     out = Reshape((outputHeight * outputWidth, nClasses))(conv10)
#     out = Activation('softmax')(out)

#     model = Model(inputs, out)
#     model.outputHeight = outputHeight
#     model.outputWidth = outputWidth

#     return model

# model1 = SEUnet(nClasses=5)
# model1.summary()

# x = model1.get_layer(index=-3).output

# # folder=["Sprout","Black_smut","Rotten","Background"]
# out0 = Conv2D(2, (1, 1), activation='softmax',name='sprout')(x)
# out1 = Conv2D(2, (1, 1), activation='softmax',name='peeled')(x)
# out2 = Conv2D(2, (1, 1), activation='softmax',name='rotten')(x)
# out3 = Conv2D(2, (1, 1), activation='softmax',name='double')(x)
# out4 = Conv2D(2, (1, 1), activation='softmax',name='background')(x)


# model_new = Model(inputs = model1.input,outputs = [out0,out1,out2,out3,out4])
# model_new.summary()

In [ ]:
# model_new.load_weights(r"/home/agrograde/Desktop/quantization/quantized/onin_28th_june_spplrobg_28thjune2024_weights.h5")

In [ ]:
import os
import tempfile`

In [ ]:
_, float_file = tempfile.mkstemp('.tflite')

In [ ]:
with open(float_file, 'wb') as f:
  f.write(float_tflite_model)

In [ ]:
print("Float model in Mb:", os.path.getsize(float_file) / float(2**20))

In [ ]:
import tensorflow_model_optimization as tfmot

In [ ]:
quantize_model = tfmot.quantization.keras.quantize_model

In [ ]:
# q_aware stands for for quantization aware.
q_aware_model = quantize_model(model_new)

In [ ]:
q_aware_model = quantize_model(model_new)

import tensorflow as tf
from tensorflow_model_optimization.python.core.quantization.keras import quantize_config
#from tensorflow_model_optimization.python.core.quantization.keras import quantize_annotate_layer
from tensorflow_model_optimization.python.core.quantization.keras import quantize_apply

In [ ]:
!pip install tensorflow-model-optimization


### train,val,test

In [ ]:
def tf_dataset(x, y1, y2, y3, y4, y5, batch_size, train_split=0.8, val_split=0.1):
    # Combine inputs and targets into a single dataset
    dataset = tf.data.Dataset.from_tensor_slices((x, y1, y2, y3, y4, y5))
    
    # Shuffle the dataset with a buffer size of 1000
    dataset = dataset.shuffle(buffer_size=1000)
    
    # Apply any necessary preprocessing to each sample (e.g., normalization, resizing)
    dataset = dataset.map(preprocess)
    
    # Calculate the number of samples for each split
    total_samples = len(x)
    train_samples = int(total_samples * train_split)
    val_samples = int(total_samples * val_split)
    
    # Split dataset into training and validation sets
    train_dataset = dataset.take(train_samples)
    val_dataset = dataset.skip(train_samples).take(val_samples)
    
    # Split inputs and targets into separate datasets
    train_images = train_dataset.map(lambda x, *y: x)
    train_sprout = train_dataset.map(lambda x, *y: y[0])
    train_peeled = train_dataset.map(lambda x, *y: y[1])
    train_rotten = train_dataset.map(lambda x, *y: y[2])
    train_double = train_dataset.map(lambda x, *y: y[3])
    train_background = train_dataset.map(lambda x, *y: y[4])
    
    # For validation dataset
    val_images = val_dataset.map(lambda x, *y: x)
    val_sprout = val_dataset.map(lambda x, *y: y[0])
    val_peeled = val_dataset.map(lambda x, *y: y[1])
    val_rotten = val_dataset.map(lambda x, *y: y[2])
    val_double = val_dataset.map(lambda x, *y: y[3])
    val_background = val_dataset.map(lambda x, *y: y[4])
    
    # Batch the datasets
    train_images = train_images.batch(batch_size)
    train_sprout = train_sprout.batch(batch_size)
    train_peeled = train_peeled.batch(batch_size)
    train_rotten = train_rotten.batch(batch_size)
    train_double = train_double.batch(batch_size)
    train_background = train_background.batch(batch_size)
    
    val_images = val_images.batch(batch_size)
    val_sprout = val_sprout.batch(batch_size)
    val_peeled = val_peeled.batch(batch_size)
    val_rotten = val_rotten.batch(batch_size)
    val_double = val_double.batch(batch_size)
    val_background = val_background.batch(batch_size)
    
    # Prefetch data for improved performance
    train_images = train_images.prefetch(buffer_size=tf.data.AUTOTUNE)
    train_sprout = train_sprout.prefetch(buffer_size=tf.data.AUTOTUNE)
    train_peeled = train_peeled.prefetch(buffer_size=tf.data.AUTOTUNE)
    train_rotten = train_rotten.prefetch(buffer_size=tf.data.AUTOTUNE)
    train_double = train_double.prefetch(buffer_size=tf.data.AUTOTUNE)
    train_background = train_background.prefetch(buffer_size=tf.data.AUTOTUNE)
    
    val_images = val_images.prefetch(buffer_size=tf.data.AUTOTUNE)
    val_sprout = val_sprout.prefetch(buffer_size=tf.data.AUTOTUNE)
    val_peeled = val_peeled.prefetch(buffer_size=tf.data.AUTOTUNE)
    val_rotten = val_rotten.prefetch(buffer_size=tf.data.AUTOTUNE)
    val_double = val_double.prefetch(buffer_size=tf.data.AUTOTUNE)
    val_background = val_background.prefetch(buffer_size=tf.data.AUTOTUNE)
    
    return (train_images, train_sprout, train_peeled, train_rotten, train_double, train_background), \
           (val_images, val_sprout, val_peeled, val_rotten, val_double, val_background)


In [ ]:
(train_images, train_sprout, train_peeled, train_rotten, train_double, train_background), \
(val_images, val_sprout, val_peeled, val_rotten, val_double, val_background) = \
    tf_dataset(images, sprout, peeled, rotten, double, background, batch_size=16, 
               train_split=0.8, val_split=0.1)

In [ ]:
import matplotlib.pyplot as plt

def visualize_batch(images_batch, masks_batch, batch_size=16):
    plt.figure(figsize=(20, 20))
    
    for i in range(batch_size):
        img = images_batch[i]
        mask_sprout = masks_batch[0][i][:, :, 1]
        mask_peeled = masks_batch[1][i][:, :, 1]
        mask_rotten = masks_batch[2][i][:, :, 1]
        mask_double = masks_batch[3][i][:, :, 1]
        mask_background = masks_batch[4][i][:, :, 1]
        
        plt.subplot(batch_size, 6, i*6 + 1)
        plt.imshow(img.astype(np.uint8))
        plt.title('Image')
        plt.axis('off')
        
        plt.subplot(batch_size, 6, i*6 + 2)
        plt.imshow(mask_sprout, cmap='gray')
        plt.title('Sprout Mask')
        plt.axis('off')
        
        plt.subplot(batch_size, 6, i*6 + 3)
        plt.imshow(mask_peeled, cmap='gray')
        plt.title('Peeled Mask')
        plt.axis('off')
        
        plt.subplot(batch_size, 6, i*6 + 4)
        plt.imshow(mask_rotten, cmap='gray')
        plt.title('Rotten Mask')
        plt.axis('off')
        
        plt.subplot(batch_size, 6, i*6 + 5)
        plt.imshow(mask_double, cmap='gray')
        plt.title('Double Mask')
        plt.axis('off')
        
        plt.subplot(batch_size, 6, i*6 + 6)
        plt.imshow(mask_background, cmap='gray')
        plt.title('Background Mask')
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()




In [ ]:
# Get the first batch from the dataset
for batch in train_images.take(1):
    images_batch = batch.numpy()

masks_batches = []
for dataset in [train_sprout, train_peeled, train_rotten, train_double, train_background]:
    for batch in dataset.take(1):
        masks_batches.append(batch.numpy())


In [ ]:

# Visualize the first batch
visualize_batch(images_batch, masks_batches)

In [ ]:
#main utility functions for model
def dsc(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return score

def dice_loss(y_true, y_pred):
    loss = 1 - dsc(y_true, y_pred)
    return loss

def bce_dice_loss(y_true, y_pred):
    loss = binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)
    return loss

def iou_score(gt, pr, class_weights=1., smooth=1, per_image=True, threshold=None):
    '''
    input：
        gt: ground truth 4D keras tensor (B, H, W, C)
        pr: prediction 4D keras tensor (B, H, W, C)
        class_weights: 1. or list of class weights, len(weights) = C
        smooth: value to avoid division by zero
        per_image: if ``True``, metric is calculated as mean over images in batch (B),
            else over whole batch
        threshold: value to round predictions (use ``>`` comparison), 
        if ``None`` prediction prediction will not be round
    output：
        IoU/Jaccard score in range [0, 1]
    '''
    if per_image:
        axes = [1, 2]
    else:
        axes = [0, 1, 2]
        
    if threshold is not None:
        pr = tf.greater(pr, threshold)
        pr = tf.cast(pr, dtype=tf.float32)

    intersection = tf.reduce_sum(gt * pr, axis=axes)
    union = tf.reduce_sum(gt + pr, axis=axes) - intersection
    iou = (intersection + smooth) / (union + smooth)

    # mean per image
    if per_image:
        iou = tf.reduce_mean(iou, axis=0)

    # weighted mean per class
    iou = tf.reduce_mean(iou * class_weights)

    return iou

In [ ]:
#build model
import segmentation_models as sm
from keras.layers import *
from keras import layers
from keras.models import Model

def SEModule(input, ratio, out_dim):
    # bs, c, h, w
    x = GlobalAveragePooling2D()(input)
    excitation = Dense(units=out_dim // ratio)(x)
    excitation = Activation('relu')(excitation)
    excitation = Dense(units=out_dim)(excitation)
    excitation = Activation('sigmoid')(excitation)
    excitation = Reshape((1, 1, out_dim))(excitation)
    scale = multiply([input, excitation])
    return scale


def SEUnet(nClasses, input_height=224, input_width=224):
    inputs = Input(shape=(input_height, input_width, 3))
    conv1 = Conv2D(16,
                   3,
                   activation='relu',
                   padding='same',
                   kernel_initializer='he_normal')(inputs)
    conv1 = BatchNormalization()(conv1)

    conv1 = Conv2D(16,
                   3,
                   activation='relu',
                   padding='same',
                   kernel_initializer='he_normal')(conv1)
    conv1 = BatchNormalization()(conv1)

    # se
    conv1 = SEModule(conv1, 4, 16)

    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(32,
                   3,
                   activation='relu',
                   padding='same',
                   kernel_initializer='he_normal')(pool1)
    conv2 = BatchNormalization()(conv2)

    conv2 = Conv2D(32,
                   3,
                   activation='relu',
                   padding='same',
                   kernel_initializer='he_normal')(conv2)
    conv2 = BatchNormalization()(conv2)

    # se
    conv2 = SEModule(conv2, 8, 32)

    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(64,
                   3,
                   activation='relu',
                   padding='same',
                   kernel_initializer='he_normal')(pool2)
    conv3 = BatchNormalization()(conv3)

    conv3 = Conv2D(64,
                   3,
                   activation='relu',
                   padding='same',
                   kernel_initializer='he_normal')(conv3)
    conv3 = BatchNormalization()(conv3)

    # se
    conv3 = SEModule(conv3, 8, 64)

    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(128,
                   3,
                   activation='relu',
                   padding='same',
                   kernel_initializer='he_normal')(pool3)
    conv4 = BatchNormalization()(conv4)

    conv4 = Conv2D(128,
                   3,
                   activation='relu',
                   padding='same',
                   kernel_initializer='he_normal')(conv4)
    conv4 = BatchNormalization()(conv4)

    # se
    conv4 = SEModule(conv4, 16, 128)

    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(256,
                   3,
                   activation='relu',
                   padding='same',
                   kernel_initializer='he_normal')(pool4)
    conv5 = BatchNormalization()(conv5)
    conv5 = Conv2D(256,
                   3,
                   activation='relu',
                   padding='same',
                   kernel_initializer='he_normal')(conv5)
    conv5 = BatchNormalization()(conv5)

    # se
    conv5 = SEModule(conv5, 16, 256)

    up6 = Conv2D(128,
                 2,
                 activation='relu',
                 padding='same',
                 kernel_initializer='he_normal')(UpSampling2D(size=(2,
                                                                    2))(conv5))
    up6 = BatchNormalization()(up6)

    merge6 = concatenate([conv4, up6], axis=3)
    conv6 = Conv2D(128,
                   3,
                   activation='relu',
                   padding='same',
                   kernel_initializer='he_normal')(merge6)
    conv6 = BatchNormalization()(conv6)

    conv6 = Conv2D(128,
                   3,
                   activation='relu',
                   padding='same',
                   kernel_initializer='he_normal')(conv6)
    conv6 = BatchNormalization()(conv6)

    # se
    conv6 = SEModule(conv6, 16, 128)

    up7 = Conv2D(64,
                 2,
                 activation='relu',
                 padding='same',
                 kernel_initializer='he_normal')(UpSampling2D(size=(2,
                                                                    2))(conv6))
    up7 = BatchNormalization()(up7)

    merge7 = concatenate([conv3, up7], axis=3)
    conv7 = Conv2D(64,
                   3,
                   activation='relu',
                   padding='same',
                   kernel_initializer='he_normal')(merge7)
    conv7 = BatchNormalization()(conv7)

    conv7 = Conv2D(64,
                   3,
                   activation='relu',
                   padding='same',
                   kernel_initializer='he_normal')(conv7)
    conv7 = BatchNormalization()(conv7)

    # se
    conv7 = SEModule(conv7, 8, 64)

    up8 = Conv2D(32,
                 2,
                 activation='relu',
                 padding='same',
                 kernel_initializer='he_normal')(UpSampling2D(size=(2,
                                                                    2))(conv7))
    up8 = BatchNormalization()(up8)

    merge8 = concatenate([conv2, up8], axis=3)
    conv8 = Conv2D(32,
                   3,
                   activation='relu',
                   padding='same',
                   kernel_initializer='he_normal')(merge8)
    conv8 = BatchNormalization()(conv8)

    conv8 = Conv2D(32,
                   3,
                   activation='relu',
                   padding='same',
                   kernel_initializer='he_normal')(conv8)
    conv8 = BatchNormalization()(conv8)

    # se
    conv8 = SEModule(conv8, 4, 32)

    up9 = Conv2D(16,
                 2,
                 activation='relu',
                 padding='same',
                 kernel_initializer='he_normal')(UpSampling2D(size=(2,
                                                                    2))(conv8))
    up9 = BatchNormalization()(up9)

    merge9 = concatenate([conv1, up9], axis=3)
    conv9 = Conv2D(16,
                   3,
                   activation='relu',
                   padding='same',
                   kernel_initializer='he_normal')(merge9)
    conv9 = BatchNormalization()(conv9)

    conv9 = Conv2D(16,
                   3,
                   activation='relu',
                   padding='same',
                   kernel_initializer='he_normal')(conv9)
    conv9 = BatchNormalization()(conv9)

    # se
    conv9 = SEModule(conv9, 2, 16)

    conv10 = Conv2D(nClasses, (3, 3), padding='same')(conv9)
    conv10 = BatchNormalization()(conv10)

    outputHeight = Model(inputs, conv10).output_shape[1]
    outputWidth = Model(inputs, conv10).output_shape[2]

    out = (Reshape((outputHeight * outputWidth, nClasses)))(conv10)
    out = Activation('softmax')(out)

    model = Model(inputs,out)
    model.outputHeight = outputHeight
    model.outputWidth = outputWidth

    return model

model1 = SEUnet(nClasses=5)
model1.summary()

In [ ]:
x = model1.get_layer(index=-3).output

# folder=["Sprout","Black_smut","Rotten","Background"]
out0 = Conv2D(2, (1, 1), activation='softmax',name='sprout')(x)
out1 = Conv2D(2, (1, 1), activation='softmax',name='peeled')(x)
out2 = Conv2D(2, (1, 1), activation='softmax',name='rotten')(x)
out3 = Conv2D(2, (1, 1), activation='softmax',name='double')(x)
out4 = Conv2D(2, (1, 1), activation='softmax',name='background')(x)


model_new = Model(inputs = model1.input,outputs = [out0,out1,out2,out3,out4])
model_new.summary()

In [ ]:
from segmentation_models.metrics import  f1_score,precision,recall
from segmentation_models.metrics import  f1_score
from tensorflow.keras.callbacks import TensorBoard
import datetime

In [ ]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:

from tensorflow.keras.optimizers import Adam
model_name = 'onin_24th_jan_spplrobg'
date = '24thdec2024'
learning_rate = 0.001
# num_epochs = 20




# Initialize the optimizer
opt = Adam(lr=learning_rate)



# Define losses, metrics, and loss weights
losses = {"sprout": sm.losses.bce_jaccard_loss, "peeled": sm.losses.bce_jaccard_loss, "rotten": sm.losses.bce_jaccard_loss,"double": sm.losses.bce_jaccard_loss,"background": sm.losses.bce_jaccard_loss}
metrics = {"sprout": [iou_score, f1_score,precision,recall], "peeled": [iou_score, f1_score,precision,recall], "rotten": [iou_score, f1_score,precision,recall],"double": [iou_score, f1_score,precision,recall], "background": [iou_score, f1_score,precision,recall]}
loss_weights = {"sprout": 1.0, "peeled": 1.0, "rotten": 1.0, "double": 1.0, "background": 2.0}

print("[INFO] compiling model...")

# Define callbacks
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

LR_Reduce_callback = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=10, verbose=1, mode='auto')
model_ckpt1 = ModelCheckpoint(model_name+'_'+date+'.h5', monitor='loss', save_weights_only=False,save_best_only=True, period=1)
model_ckpt2 = ModelCheckpoint(model_name+'_'+date+'_weights.h5', monitor='loss', save_weights_only=True,save_best_only=True, period=1)
# reduce_rl_plateau = CustomReduceLRoP(patience=4, 
#                               verbose=1, 
#                               optim_lr=opt.learning_rate, 
#                               mode='auto',
#                               factor=0.1)
# Compile the model using the initialized optimizer
model_new.compile(optimizer=opt, loss=losses, metrics=metrics)

### for training and validation and testing

In [ ]:
labels_dict = {
    "sprout": train_sprout,
    "peeled": train_peeled,
    "rotten": train_rotten,
    "double": train_double,
    "background": train_background
}
val_labels_dict = {
    "sprout": val_sprout,
    "peeled": val_peeled,
    "rotten": val_rotten,
    "double": val_double,
    "background": val_background
}
val_data = tf.data.Dataset.zip((val_images, val_labels_dict)) 
# Zip images and label dictionary into a single dataset
train_data = tf.data.Dataset.zip((train_images, labels_dict))


In [ ]:

H = model_new.fit(train_data,
                  epochs=2,
                  validation_data=val_data,
                  callbacks=[tensorboard_callback, LR_Reduce_callback, model_ckpt1, model_ckpt2],
                  batch_size=16)

In [ ]:
%load_ext tensorboard         
%tensorboard --logdir logs/fit

In [ ]:
#plot graphs
# folder=['holes":1.0,"potato": 1.0,"rotten": 1.0,"cuts"]
fig, (ax0,ax1, ax2, ax3, ax4,ax5) = plt.subplots(1, 6, figsize = (41, 10))
# fig, (ax0,ax1, ax2) = plt.subplots(1, 3, figsize = (17, 5))

ax0.plot(H.history['loss'], '-', label = 'Loss')
ax0.plot(H.history['val_loss'], '-', label = 'Validation Loss')
ax0.legend()
plt.savefig('loss.png')

ax1.plot(100*np.array(H.history['Sprout_iou_score']), '-', 
         label = 'Accuracy_Sprout')
ax1.plot(100*np.array(H.history['val_Sprout_iou_score']), '-',
         label = 'Validation Accuracy_Sprout')
ax1.legend()


ax4.plot(100*np.array(H.history['Peeled_iou_score']), '-', 
         label = 'Accuracy_Peeled')
ax4.plot(100*np.array(H.history['val_Peeled_iou_score']), '-',
         label = 'Validation Accuracy_Peeled')
ax4.legend()



ax2.plot(100*np.array(H.history['Rotten_iou_score']), '-', 
         label = 'Accuracy_Rotten')
ax2.plot(100*np.array(H.history['val_Rotten_iou_score']), '-',
         label = 'Validation Accuracy_Rotten')
ax2.legend()


ax3.plot(100*np.array(H.history['Background_iou_score']), '-', 
         label = 'Accuracy_Background')
ax3.plot(100*np.array(H.history['val_Background_iou_score']), '-',
         label = 'Validation Accuracy_Background')
ax3.legend()





# ax4.plot(100*np.array(H.history['Peeled_iou_score']), '-', 
#          label = 'Accuracy_Peeled')
# ax4.plot(100*np.array(H.history['val_Peeled_iou_score']), '-',
#          label = 'Validation Accuracy_Peeled')
# ax4.legend()

In [ ]:
import pandas as pd
matrix_dict = {"train_loss":[],"val_loss":[],train.....}

In [ ]:
import tensorflow as tf
from tqdm import tqdm

train_losses = []
train_ious_sprout = []
train_ious_peeled = []
train_ious_rotten = []
train_ious_double = []
train_ious_background = []

train_f1_scores_sprout = []
train_f1_scores_peeled = []
train_f1_scores_rotten = []
train_f1_scores_double = []
train_f1_scores_background = []


val_losses = []
val_ious_sprout = []
val_ious_peeled = []
val_ious_rotten = []
val_ious_double = []
val_ious_background = []

val_f1_scores_sprout = []
val_f1_scores_peeled = []
val_f1_scores_rotten = []
val_f1_scores_double = []
val_f1_scores_background = []

train_precisions_sprout = []
train_precisions_peeled = []
train_precisions_rotten = []
train_precisions_double = []
train_precisions_background = []

train_recalls_sprout = []
train_recalls_peeled = []
train_recalls_rotten = []
train_recalls_double = []
train_recalls_background = []

val_precisions_sprout = []
val_precisions_peeled = []
val_precisions_rotten = []
val_precisions_double = []
val_precisions_background = []

val_recalls_sprout = []
val_recalls_peeled = []
val_recalls_rotten = []
val_recalls_double = []
val_recalls_background = []






# Training loop
for epoch in tqdm(range(num_epochs), desc="Epochs"):
    print("Epoch {}/{}".format(epoch + 1, num_epochs))
    
    # Initialize the training metrics
    train_loss = 0.0
    train_iou_sprout = 0.0
    train_iou_peeled = 0.0
    train_iou_rotten = 0.0
    train_iou_double = 0.0
    train_iou_background = 0.0
    train_f1_score_sprout = 0.0
    train_f1_score_peeled = 0.0
    train_f1_score_rotten = 0.0
    train_f1_score_double = 0.0
    train_f1_score_background = 0.0
    train_precision_sprout = 0.0
    train_precision_peeled = 0.0
    train_precision_rotten = 0.0
    train_precision_double = 0.0
    train_precision_background = 0.0
    train_recall_sprout = 0.0
    train_recall_peeled = 0.0
    train_recall_rotten = 0.0
    train_recall_double = 0.0
    train_recall_background = 0.0
    num_batches = 0
    
    # Iterate over the training dataset
    for step, (images, sprout_masks, peeled_masks, rotten_masks, double_masks, background_masks) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            sprout_pred, peeled_pred, rotten_pred, double_pred, background_pred = model_new(images, training=True)
            sprout_loss = sm.losses.bce_jaccard_loss(sprout_masks, sprout_pred)
            peeled_loss = sm.losses.bce_jaccard_loss(peeled_masks, peeled_pred)
            rotten_loss = sm.losses.bce_jaccard_loss(rotten_masks, rotten_pred)
            double_loss = sm.losses.bce_jaccard_loss(double_masks, double_pred)
            background_loss = sm.losses.bce_jaccard_loss(background_masks, background_pred)


            iou_sprout = iou_score(sprout_masks, sprout_pred)
            iou_peeled = iou_score(peeled_masks, peeled_pred)
            iou_rotten = iou_score(rotten_masks, rotten_pred)
            iou_double = iou_score(double_masks, double_pred)
            iou_background = iou_score(background_masks, background_pred)
            
            f1_sprout = f1_score(sprout_masks, sprout_pred)
            f1_peeled = f1_score(peeled_masks, peeled_pred)
            f1_rotten = f1_score(rotten_masks, rotten_pred)
            f1_double = f1_score(double_masks, double_pred)
            f1_background = f1_score(background_masks, background_pred)
            
            precision_sprout = precision(sprout_masks, sprout_pred)
            precision_peeled = precision(peeled_masks, peeled_pred)
            precision_rotten = precision(rotten_masks, rotten_pred)
            precision_double = precision(double_masks, double_pred)
            precision_background = precision(background_masks, background_pred)
            
            recall_sprout = recall(sprout_masks, sprout_pred)
            recall_peeled = recall(peeled_masks, peeled_pred)
            recall_rotten = recall(rotten_masks, rotten_pred)
            recall_double = recall(double_masks, double_pred)
            recall_background = recall(background_masks, background_pred)
            




            

            total_loss = sprout_loss + peeled_loss + rotten_loss + double_loss + background_loss

        gradients = tape.gradient(total_loss, model_new.trainable_variables)
        opt.apply_gradients(zip(gradients, model_new.trainable_variables))

        
        train_loss += total_loss
        train_iou_sprout += iou_sprout
        train_iou_peeled += iou_peeled
        train_iou_rotten += iou_rotten
        train_iou_double += iou_double
        train_iou_background += iou_background

        train_f1_score_sprout += f1_sprout
        train_f1_score_peeled += f1_peeled
        train_f1_score_rotten += f1_rotten
        train_f1_score_double += f1_double
        train_f1_score_background += f1_background

        train_precision_sprout += precision_sprout
        train_precision_peeled += precision_peeled
        train_precision_rotten += precision_rotten
        train_precision_double += precision_double
        train_precision_background += precision_background

        train_recall_sprout += recall_sprout
        train_recall_peeled += recall_peeled
        train_recall_rotten += recall_rotten
        train_recall_double += recall_double
        train_recall_background += recall_background

        if (step + 1) % 40 == 0:
            
            print("Epoch {}/{}, Iteration {}/{} - train_Loss: {:.4f}, train_IOU Sprout: {:.4f}, train_IOU Peeled: {:.4f}, train_IOU Rotten: {:.4f}, train_IOU Double: {:.4f}, train_IOU Background: {:.4f}, train_F1 Score Sprout: {:.4f}, train_F1 Score Peeled: {:.4f}, train_F1 Score Rotten: {:.4f}, train_F1 Score Double: {:.4f}, train_F1 Score Background: {:.4f}, train_Precision Sprout: {:.4f}, train_Precision Peeled: {:.4f}, train_Precision Rotten: {:.4f}, train_Precision Double: {:.4f}, train_Precision Background: {:.4f}, train_Recall Sprout: {:.4f}, train_Recall Peeled: {:.4f}, train_Recall Rotten: {:.4f}, train_Recall double: {:.4f}, train_Recall Background: {:.4f}".format(epoch + 1, num_epochs, step + 1, len(train_dataset), total_loss.numpy(), iou_sprout,iou_peeled,iou_rotten,iou_double,iou_background,f1_sprout,f1_peeled,f1_rotten,f1_double,f1_background,precision_sprout,precision_peeled,precision_rotten, precision_double,precision_background,recall_sprout,recall_peeled,recall_rotten,recall_double,recall_background))
       
        
    


        
        num_batches += 1
    
    # Calculate average metrics for the epoch
    train_loss /= num_batches
    train_iou_sprout /= num_batches
    train_iou_peeled /= num_batches
    train_iou_rotten /= num_batches
    train_iou_double /= num_batches
    train_iou_background /= num_batches
    
    train_f1_score_sprout /= num_batches
    train_f1_score_peeled /= num_batches
    train_f1_score_rotten /= num_batches
    train_f1_score_double /= num_batches
    train_f1_score_background /= num_batches

    train_precision_sprout /= num_batches
    train_precision_peeled /= num_batches
    train_precision_rotten /= num_batches
    train_precision_double /= num_batches
    train_precision_background /= num_batches

    train_recall_sprout /= num_batches
    train_recall_peeled /= num_batches
    train_recall_rotten /= num_batches
    train_recall_double /= num_batches
    train_recall_background /= num_batches

     # Append the values to the respective lists
    train_losses.append(train_loss)
    train_ious_sprout.append(train_iou_sprout)
    train_ious_peeled.append(train_iou_peeled)
    train_ious_rotten.append(train_iou_rotten)
    train_ious_double.append(train_iou_double)
    train_ious_background.append(train_iou_background)
    
    train_f1_scores_sprout.append(train_f1_score_sprout)
    train_f1_scores_peeled.append(train_f1_score_peeled)
    train_f1_scores_rotten.append(train_f1_score_rotten)
    train_f1_scores_double.append(train_f1_score_double)
    train_f1_scores_background.append(train_f1_score_background)
    
    train_precisions_sprout.append(train_precision_sprout)
    train_precisions_peeled.append(train_precision_peeled)
    train_precisions_rotten.append(train_precision_rotten)
    train_precisions_double.append(train_precision_double)
    train_precisions_background.append(train_precision_background)
    
    train_recalls_sprout.append(train_recall_sprout)
    train_recalls_peeled.append(train_recall_peeled)
    train_recalls_rotten.append(train_recall_rotten)
    train_recalls_double.append(train_recall_double)
    train_recalls_background.append(train_recall_background)

    


    print("Training Loss: {:.4f}, Train_IOU Sprout: {:.4f}, Train_IOU Peeled: {:.4f}, Train_IOU Rotten: {:.4f}, Train_IOU Double: {:.4f}, Train_IOU Background: {:.4f},Train_F1 Score Sprout: {:.4f}, Train_F1 Score Peeled: {:.4f}, Train_F1 Score Rotten: {:.4f}, Train_F1 Score Double: {:.4f}, Train_F1 Score Background: {:.4f},Train_Precision Sprout: {:.4f},Train_Precision Peeled: {:.4f}, Train_Precision Rotten: {:.4f},Train_Precision Double: {:.4f},Train_Precision Background: {:.4f},Train_Recall Sprout: {:.4f},Train_Recall Peeled: {:.4f},Train_Recall Rotten: {:.4f},Train_Recall Double: {:.4f},Train_Recall Background: {:.4f}".format(train_loss, train_iou_sprout, train_iou_peeled, train_iou_rotten, train_iou_double, train_iou_background,train_f1_score_sprout, train_f1_score_peeled, train_f1_score_rotten, train_f1_score_double, train_f1_score_background,train_precision_sprout, train_precision_peeled, train_precision_rotten, train_precision_double, train_precision_background,train_recall_sprout, train_recall_peeled, train_recall_rotten, train_recall_double, train_recall_background))


    val_loss = 0.0
    val_iou_sprout = 0.0
    val_iou_peeled = 0.0
    val_iou_rotten = 0.0
    val_iou_double = 0.0
    val_iou_background = 0.0
    val_f1_score_sprout = 0.0
    val_f1_score_peeled = 0.0
    val_f1_score_rotten = 0.0
    val_f1_score_double = 0.0
    val_f1_score_background = 0.0
    val_precision_sprout = 0.0
    val_precision_peeled = 0.0
    val_precision_rotten = 0.0
    val_precision_double = 0.0
    val_precision_background = 0.0
    val_recall_sprout = 0.0
    val_recall_peeled = 0.0
    val_recall_rotten = 0.0
    val_recall_double = 0.0
    val_recall_background = 0.0
    num_val_batches = 0
    
    for val_images, val_sprout_masks, val_peeled_masks, val_rotten_masks, val_double_masks, val_background_masks in val_dataset:
        # Predict
        val_sprout_pred, val_peeled_pred, val_rotten_pred, val_double_pred, val_background_pred = model_new(val_images, training=False)

        # Compute the loss for each output
        val_sprout_loss = sm.losses.bce_jaccard_loss(val_sprout_masks, val_sprout_pred)
        val_peeled_loss = sm.losses.bce_jaccard_loss(val_peeled_masks, val_peeled_pred)
        val_rotten_loss = sm.losses.bce_jaccard_loss(val_rotten_masks, val_rotten_pred)
        val_double_loss = sm.losses.bce_jaccard_loss(val_double_masks, val_double_pred)
        val_background_loss = sm.losses.bce_jaccard_loss(val_background_masks, val_background_pred)

        # Compute total loss
        val_total_loss = (val_sprout_loss + val_peeled_loss + val_rotten_loss + val_double_loss + val_background_loss).numpy()

        # Update validation metrics
        val_loss += val_total_loss
        val_iou_sprout += iou_score(val_sprout_masks, val_sprout_pred)
        val_iou_peeled += iou_score(val_peeled_masks, val_peeled_pred)
        val_iou_rotten += iou_score(val_rotten_masks, val_rotten_pred)
        val_iou_double += iou_score(val_double_masks, val_double_pred)
        val_iou_background += iou_score(val_background_masks, val_background_pred)
        
        val_f1_score_sprout += f1_score(val_sprout_masks, val_sprout_pred)
        val_f1_score_peeled += f1_score(val_peeled_masks, val_peeled_pred)
        val_f1_score_rotten += f1_score(val_rotten_masks, val_rotten_pred)
        val_f1_score_double += f1_score(val_double_masks, val_double_pred)
        val_f1_score_background += f1_score(val_background_masks, val_background_pred)
        
        val_precision_sprout += precision(val_sprout_masks, val_sprout_pred)
        val_precision_peeled += precision(val_peeled_masks, val_peeled_pred)
        val_precision_rotten += precision(val_rotten_masks, val_rotten_pred)
        val_precision_double += precision(val_double_masks, val_double_pred)
        val_precision_background += precision(val_background_masks, val_background_pred)
        
        val_recall_sprout += recall(val_sprout_masks, val_sprout_pred)
        val_recall_peeled += recall(val_peeled_masks, val_peeled_pred)
        val_recall_rotten += recall(val_rotten_masks, val_rotten_pred)
        val_recall_double += recall(val_double_masks, val_double_pred)
        val_recall_background += recall(val_background_masks, val_background_pred)

        num_val_batches += 1

        if (step + 1) % 40 == 0:
            print("Epoch {}/{}, Iteration {}/{} - val_Loss: {:.4f}, val_IOU Sprout: {:.4f}, val_IOU Peeled: {:.4f}, val_IOU Rotten: {:.4f}, val_IOU Double: {:.4f}, val_IOU Background: {:.4f}, val_F1 Score Sprout: {:.4f}, val_F1 Score Peeled: {:.4f}, val_F1 Score Rotten: {:.4f}, val_F1 Score Double: {:.4f}, val_F1 Score Background: {:.4f}, val_Precision Sprout: {:.4f}, val_Precision Peeled: {:.4f}, val_Precision Rotten: {:.4f}, val_Precision Double: {:.4f}, val_Precision Background: {:.4f}, val_Recall Sprout: {:.4f}, val_Recall Peeled: {:.4f}, val_Recall Rotten: {:.4f}, val_Recall Double: {:.4f}, val_Recall Background: {:.4f}".format(epoch + 1, num_epochs, step + 1, len(train_dataset), val_loss, val_iou_sprout, val_iou_peeled, val_iou_rotten, val_iou_double, val_iou_background, val_f1_score_sprout, val_f1_score_peeled, val_f1_score_rotten, val_f1_score_double, val_f1_score_background, val_precision_sprout, val_precision_peeled, val_precision_rotten, val_precision_double, val_precision_background, val_recall_sprout, val_recall_peeled, val_recall_rotten, val_recall_double, val_recall_background))

            
 
                                    
            



    
    # Calculate average validation metrics
    val_loss /= num_val_batches
    val_iou_sprout /= num_val_batches
    val_iou_peeled /= num_val_batches
    val_iou_rotten /= num_val_batches
    val_iou_double /= num_val_batches
    val_iou_background /= num_val_batches
    
    val_f1_score_sprout /= num_val_batches
    val_f1_score_peeled /= num_val_batches
    val_f1_score_rotten /= num_val_batches
    val_f1_score_double /= num_val_batches
    val_f1_score_background /= num_val_batches
    
    val_precision_sprout /= num_val_batches
    val_precision_peeled /= num_val_batches
    val_precision_rotten /= num_val_batches
    val_precision_double /= num_val_batches
    val_precision_background /= num_val_batches
    
    val_recall_sprout /= num_val_batches
    val_recall_peeled /= num_val_batches
    val_recall_rotten /= num_val_batches
    val_recall_double /= num_val_batches
    val_recall_background /= num_val_batches


    val_losses.append(val_loss)
    val_ious_sprout.append(val_iou_sprout)
    val_ious_peeled.append(val_iou_peeled)
    val_ious_rotten.append(val_iou_rotten)
    val_ious_double.append(val_iou_double)
    val_ious_background.append(val_iou_background)
    
    val_f1_scores_sprout.append(val_f1_score_sprout)
    val_f1_scores_peeled.append(val_f1_score_peeled)
    val_f1_scores_rotten.append(val_f1_score_rotten)
    val_f1_scores_double.append(val_f1_score_double)
    val_f1_scores_background.append(val_f1_score_background)
    
    val_precisions_sprout.append(val_precision_sprout)
    val_precisions_peeled.append(val_precision_peeled)
    val_precisions_rotten.append(val_precision_rotten)
    val_precisions_double.append(val_precision_double)
    val_precisions_background.append(val_precision_background)
    
    val_recalls_sprout.append(val_recall_sprout)
    val_recalls_peeled.append(val_recall_peeled)
    val_recalls_rotten.append(val_recall_rotten)
    val_recalls_double.append(val_recall_double)
    val_recalls_background.append(val_recall_background)



    reduce_rl_plateau.on_epoch_end(epoch, val_loss)

    current_lr = opt.learning_rate.numpy()
    print("Learning Rate:", current_lr)


    print("Validation Loss: {:.4f}, Val_IOU Sprout: {:.4f}, Val_IOU Peeled: {:.4f}, Val_IOU Rotten: {:.4f}, Val_IOU Double: {:.4f}, Val_IOU Background: {:.4f}, Val_F1 Score Sprout: {:.4f}, Val_F1 Score Peeled: {:.4f}, Val_F1 Score Rotten: {:.4f}, Val_F1 Score Double: {:.4f}, Val_F1 Score Background: {:.4f}, Val_Precision Sprout: {:.4f}, Val_Precision Peeled: {:.4f}, Val_Precision Rotten: {:.4f}, Val_Precision Double: {:.4f}, Val_Precision Background: {:.4f}, Val_Recall Sprout: {:.4f}, Val_Recall Peeled: {:.4f}, Val_Recall Rotten: {:.4f}, Val_Recall Double: {:.4f}, Val_Recall Background: {:.4f}".format(val_loss, val_iou_sprout, val_iou_peeled, val_iou_rotten, val_iou_double, val_iou_background,val_f1_score_sprout, val_f1_score_peeled, val_f1_score_rotten, val_f1_score_double, val_f1_score_background,val_precision_sprout, val_precision_peeled, val_precision_rotten, val_precision_double, val_precision_background,val_recall_sprout, val_recall_peeled, val_recall_rotten, val_recall_double, val_recall_background))
    model_new.save("model_epoch_{}.h5".format(epoch+1))


### Visualization

In [ ]:
plt.figure(figsize=(10, 5))
epochs = range(1, num_epochs + 1)
plt.plot(epochs, train_losses, label='Training Loss')
plt.plot(epochs, val_losses, label='Validation Loss')
plt.title('Training and Validation Loss per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(epochs, train_ious_sprout, label='Training IOU Sprout')
plt.plot(epochs, val_ious_sprout, label='Validation IOU Sprout')
plt.title('IOU for Sprout per Epoch')
plt.xlabel('Epoch')
plt.ylabel('IOU')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(epochs, train_ious_peeled, label='Training IOU Peeled')
plt.plot(epochs, val_ious_peeled, label='Validation IOU Peeled')
plt.title('IOU for Peeled per Epoch')
plt.xlabel('Epoch')
plt.ylabel('IOU')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(epochs, train_ious_rotten, label='Training IOU Rotten')
plt.plot(epochs, val_ious_rotten, label='Validation IOU Rotten')
plt.title('IOU for Rotten per Epoch')
plt.xlabel('Epoch')
plt.ylabel('IOU')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(epochs, train_ious_double, label='Training IOU Double')
plt.plot(epochs, val_ious_double, label='Validation IOU Double')
plt.title('IOU for Double per Epoch')
plt.xlabel('Epoch')
plt.ylabel('IOU')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(epochs, train_ious_background, label='Training IOU Background')
plt.plot(epochs, val_ious_background, label='Validation IOU Background')
plt.title('IOU for Background per Epoch')
plt.xlabel('Epoch')
plt.ylabel('IOU')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(epochs, train_precisions_sprout, label='Training Precision Sprout')
plt.plot(epochs, val_precisions_sprout, label='Validation Precision Sprout')
plt.title('Precision for Sprout per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Precision')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(epochs, train_precisions_peeled, label='Training Precision Peeled')
plt.plot(epochs, val_precisions_peeled, label='Validation Precision Peeled')
plt.title('Precision for Peeled per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Precision')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(epochs, train_precisions_rotten, label='Training Precision Rotten')
plt.plot(epochs, val_precisions_rotten, label='Validation Precision Rotten')
plt.title('Precision for Rotten per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Precision')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(epochs, train_precisions_double, label='Training Precision Double')
plt.plot(epochs, val_precisions_double, label='Validation Precision Double')
plt.title('Precision for double per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Precision')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(epochs, train_precisions_background, label='Training Precision Background')
plt.plot(epochs, val_precisions_background, label='Validation Precision Background')
plt.title('Precision for Background per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Precision')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(epochs, train_recalls_sprout, label='Training Recall Sprout')
plt.plot(epochs, val_recalls_sprout, label='Validation Recall Sprout')
plt.title('Recall for Sprout per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Recall')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(epochs, train_recalls_peeled, label='Training Recall Peeled')
plt.plot(epochs, val_recalls_peeled, label='Validation Recall Peeled')
plt.title('Recall for Peeled per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Recall')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(epochs, train_recalls_rotten, label='Training Recall Rotten')
plt.plot(epochs, val_recalls_rotten, label='Validation Recall Rotten')
plt.title('Recall for Rotten per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Recall')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(epochs, train_recalls_double, label='Training Recall Double')
plt.plot(epochs, val_recalls_double, label='Validation Recall Double')
plt.title('Recall for Double per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Recall')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(epochs, train_recalls_background, label='Training Recall Background')
plt.plot(epochs, val_recalls_background, label='Validation Recall Background')
plt.title('Recall for Background per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Recall')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(epochs, train_f1_scores_sprout, label='Training F1 Score Sprout')
plt.plot(epochs, val_f1_scores_sprout, label='Validation F1 Score Sprout')
plt.title('F1 Score for Sprout per Epoch')
plt.xlabel('Epoch')
plt.ylabel('F1 Score')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(epochs, train_f1_scores_peeled, label='Training F1 Score Peeled')
plt.plot(epochs, val_f1_scores_peeled, label='Validation F1 Score Peeled')
plt.title('F1 Score for Peeled per Epoch')
plt.xlabel('Epoch')
plt.ylabel('F1 Score')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(epochs, train_f1_scores_rotten, label='Training F1 Score Rotten')
plt.plot(epochs, val_f1_scores_rotten, label='Validation F1 Score Rotten')
plt.title('F1 Score for Rotten per Epoch')
plt.xlabel('Epoch')
plt.ylabel('F1 Score')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(epochs, train_f1_scores_double, label='Training F1 Score double')
plt.plot(epochs, val_f1_scores_double, label='Validation F1 Score double')
plt.title('F1 Score for Double per Epoch')
plt.xlabel('Epoch')
plt.ylabel('F1 Score')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(epochs, train_f1_scores_background, label='Training F1 Score Background')
plt.plot(epochs, val_f1_scores_background, label='Validation F1 Score Background')
plt.title('F1 Score for Background per Epoch')
plt.xlabel('Epoch')
plt.ylabel('F1 Score')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import time

epochs = 2
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))
    start_time = time.time()

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch_train, training=True)
            loss_value = loss_fn(y_batch_train, logits)
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # Update training metric.
        train_acc_metric.update_state(y_batch_train, logits)

        # Log every 200 batches.
        if step % 200 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )
            print("Seen so far: %d samples" % ((step + 1) * batch_size))

    # Display metrics at the end of each epoch.
    train_acc = train_acc_metric.result()
    print("Training acc over epoch: %.4f" % (float(train_acc),))

    # Reset training metrics at the end of each epoch
    train_acc_metric.reset_states()

    # Run a validation loop at the end of each epoch.
    for x_batch_val, y_batch_val in val_dataset:
        val_logits = model(x_batch_val, training=False)
        # Update val metrics
        val_acc_metric.update_state(y_batch_val, val_logits)
    val_acc = val_acc_metric.result()
    val_acc_metric.reset_states()
    print("Validation acc: %.4f" % (float(val_acc),))
    print("Time taken: %.2fs" % (time.time() - start_time))

In [ ]:
import numpy as np
import cv2
import os
from tqdm import tqdm as tqdm
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.losses import binary_crossentropy
from keras import backend as K
import segmentation_models as sm
# from keras.utils import to_categorical
from tensorflow.keras.utils import to_categorical
from tensorflow.python.keras.utils import generic_utils
from segmentation_models.losses import CategoricalFocalLoss
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.utils import to_categorical

from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau,ModelCheckpoint
# from keras.optimizers import Adam,SGD
from keras.losses import BinaryCrossentropy
from keras.metrics import MeanIoU
from glob import glob

def preprocessMasks(mask,height,width):
    mask_resized = cv2.threshold(cv2.resize(mask, (height,width)), 50, 1, cv2.THRESH_BINARY)[1]
    mask_data = np.zeros((height,width,2))
    
    for i in range(height):
        for j in range(width):


            #for segmentation mask
            if mask_resized[i,j]> 0:
                mask_data[i,j,1] = 1
            else:
                mask_data[i,j,0] = 1
                
    return mask_data       #output from the function(height, width, 2)
def load_data(path):
    images = sorted(glob(os.path.join(image_path, 'images/*')))
    sprout = sorted(glob(os.path.join(image_path, 'Sprout/*')))
    peeled = sorted(glob(os.path.join(image_path, 'Peeled/*')))
    rotten = sorted(glob(os.path.join(image_path, 'Rotten/*')))
    double = sorted(glob(os.path.join(image_path, 'Double/*')))
    background = sorted(glob(os.path.join(image_path, 'Background/*')))
    return images,sprout,peeled,rotten,double,background
    
image_path = '/home/agrograde/Desktop/4th_mar/image_path'
images,sprout,peeled,rotten,double,background = load_data(image_path)
def read_image(path):
    
    x = cv2.imread(path)
    x = cv2.resize(x, (224, 224))
    x = x.astype(np.float32)
    return x
    
def read_mask(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = preprocessMasks(x, 224, 224)
    x = x.astype(np.float32)
    return x
    
    x = read_image(images[0])
y1 = read_mask(sprout[0])
y2 = read_mask(peeled[0])
y3 = read_mask(rotten[0])
y4 = read_mask(double[0])
y5 = read_mask(background[0])
def preprocess(x,y1,y2,y3,y4,y5):
    def f(x,y1,y2,y3,y4,y5):
        x = x.decode()
        y1 = y1.decode()
        y2 = y2.decode()
        y3 = y3.decode()
        y4 = y4.decode()
        y5 = y5.decode()
        
        x = read_image(x)
        y1 = read_mask(y1)
        y2 = read_mask(y2)
        y3 = read_mask(y3)
        y4 = read_mask(y4)
        y5 = read_mask(y5)
        
        return x,y1,y2,y3,y4,y5
    
    images, sprout, peeled, rotten,double,background = tf.numpy_function(f, [x,y1,y2,y3,y4,y5], [tf.float32, tf.float32,tf.float32,tf.float32,tf.float32,tf.float32])
    images.set_shape([224, 224, 3])
    sprout.set_shape([224, 224, 2])
    peeled.set_shape([224, 224, 2])
    rotten.set_shape([224, 224, 2])
    double.set_shape([224, 224, 2])
    background.set_shape([224, 224, 2])

    return images, sprout, peeled, rotten,double, background

def tf_dataset(x, y1, y2, y3, y4, y5, batch_size, train_split=0.8, val_split=0.1):
    # Combine inputs and targets into a single dataset
    dataset = tf.data.Dataset.from_tensor_slices((x, y1, y2, y3, y4, y5))
    
    # Shuffle the dataset with a buffer size of 1000
    dataset = dataset.shuffle(buffer_size=1000)
    
    # Apply any necessary preprocessing to each sample (e.g., normalization, resizing)
    dataset = dataset.map(preprocess)
    
    # Calculate the number of samples for each split
    total_samples = len(x)
    train_samples = int(total_samples * train_split)
    val_samples = int(total_samples * val_split)
    
    # Split dataset into training and validation sets
    train_dataset = dataset.take(train_samples)
    val_dataset = dataset.skip(train_samples).take(val_samples)
    
    # Split inputs and targets into separate datasets
    train_images = train_dataset.map(lambda x, *y: x)
    train_sprout = train_dataset.map(lambda x, *y: y[0])
    train_peeled = train_dataset.map(lambda x, *y: y[1])
    train_rotten = train_dataset.map(lambda x, *y: y[2])
    train_double = train_dataset.map(lambda x, *y: y[3])
    train_background = train_dataset.map(lambda x, *y: y[4])
    
    # For validation dataset
    val_images = val_dataset.map(lambda x, *y: x)
    val_sprout = val_dataset.map(lambda x, *y: y[0])
    val_peeled = val_dataset.map(lambda x, *y: y[1])
    val_rotten = val_dataset.map(lambda x, *y: y[2])
    val_double = val_dataset.map(lambda x, *y: y[3])
    val_background = val_dataset.map(lambda x, *y: y[4])
    
    # Batch the datasets
    train_images = train_images.batch(batch_size)
    train_sprout = train_sprout.batch(batch_size)
    train_peeled = train_peeled.batch(batch_size)
    train_rotten = train_rotten.batch(batch_size)
    train_double = train_double.batch(batch_size)
    train_background = train_background.batch(batch_size)
    
    val_images = val_images.batch(batch_size)
    val_sprout = val_sprout.batch(batch_size)
    val_peeled = val_peeled.batch(batch_size)
    val_rotten = val_rotten.batch(batch_size)
    val_double = val_double.batch(batch_size)
    val_background = val_background.batch(batch_size)
    
    # Prefetch data for improved performance
    train_images = train_images.prefetch(buffer_size=tf.data.AUTOTUNE)
    train_sprout = train_sprout.prefetch(buffer_size=tf.data.AUTOTUNE)
    train_peeled = train_peeled.prefetch(buffer_size=tf.data.AUTOTUNE)
    train_rotten = train_rotten.prefetch(buffer_size=tf.data.AUTOTUNE)
    train_double = train_double.prefetch(buffer_size=tf.data.AUTOTUNE)
    train_background = train_background.prefetch(buffer_size=tf.data.AUTOTUNE)
    
    val_images = val_images.prefetch(buffer_size=tf.data.AUTOTUNE)
    val_sprout = val_sprout.prefetch(buffer_size=tf.data.AUTOTUNE)
    val_peeled = val_peeled.prefetch(buffer_size=tf.data.AUTOTUNE)
    val_rotten = val_rotten.prefetch(buffer_size=tf.data.AUTOTUNE)
    val_double = val_double.prefetch(buffer_size=tf.data.AUTOTUNE)
    val_background = val_background.prefetch(buffer_size=tf.data.AUTOTUNE)
    
    return (train_images, train_sprout, train_peeled, train_rotten, train_double, train_background), \
           (val_images, val_sprout, val_peeled, val_rotten, val_double, val_background)

(train_images, train_sprout, train_peeled, train_rotten, train_double, train_background), \
(val_images, val_sprout, val_peeled, val_rotten, val_double, val_background) = \
    tf_dataset(images, sprout, peeled, rotten, double, background, batch_size=16, 
               train_split=0.8, val_split=0.1)
#main utility functions for model
def dsc(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return score

def dice_loss(y_true, y_pred):
    loss = 1 - dsc(y_true, y_pred)
    return loss

def bce_dice_loss(y_true, y_pred):
    loss = binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)
    return loss

def iou_score(gt, pr, class_weights=1., smooth=1, per_image=True, threshold=None):
    '''
    input：
        gt: ground truth 4D keras tensor (B, H, W, C)
        pr: prediction 4D keras tensor (B, H, W, C)
        class_weights: 1. or list of class weights, len(weights) = C
        smooth: value to avoid division by zero
        per_image: if ``True``, metric is calculated as mean over images in batch (B),
            else over whole batch
        threshold: value to round predictions (use ``>`` comparison), 
        if ``None`` prediction prediction will not be round
    output：
        IoU/Jaccard score in range [0, 1]
    '''
    if per_image:
        axes = [1, 2]
    else:
        axes = [0, 1, 2]
        
    if threshold is not None:
        pr = tf.greater(pr, threshold)
        pr = tf.cast(pr, dtype=tf.float32)

    intersection = tf.reduce_sum(gt * pr, axis=axes)
    union = tf.reduce_sum(gt + pr, axis=axes) - intersection
    iou = (intersection + smooth) / (union + smooth)

    # mean per image
    if per_image:
        iou = tf.reduce_mean(iou, axis=0)

    # weighted mean per class
    iou = tf.reduce_mean(iou * class_weights)

    return iou
#build model
import segmentation_models as sm
from keras.layers import *
from keras import layers
from keras.models import Model

def SEModule(input, ratio, out_dim):
    # bs, c, h, w
    x = GlobalAveragePooling2D()(input)
    excitation = Dense(units=out_dim // ratio)(x)
    excitation = Activation('relu')(excitation)
    excitation = Dense(units=out_dim)(excitation)
    excitation = Activation('sigmoid')(excitation)
    excitation = Reshape((1, 1, out_dim))(excitation)
    scale = multiply([input, excitation])
    return scale


def SEUnet(nClasses, input_height=224, input_width=224):
    inputs = Input(shape=(input_height, input_width, 3))
    conv1 = Conv2D(16,
                   3,
                   activation='relu',
                   padding='same',
                   kernel_initializer='he_normal')(inputs)
    conv1 = BatchNormalization()(conv1)

    conv1 = Conv2D(16,
                   3,
                   activation='relu',
                   padding='same',
                   kernel_initializer='he_normal')(conv1)
    conv1 = BatchNormalization()(conv1)

    # se
    conv1 = SEModule(conv1, 4, 16)

    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(32,
                   3,
                   activation='relu',
                   padding='same',
                   kernel_initializer='he_normal')(pool1)
    conv2 = BatchNormalization()(conv2)

    conv2 = Conv2D(32,
                   3,
                   activation='relu',
                   padding='same',
                   kernel_initializer='he_normal')(conv2)
    conv2 = BatchNormalization()(conv2)

    # se
    conv2 = SEModule(conv2, 8, 32)

    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(64,
                   3,
                   activation='relu',
                   padding='same',
                   kernel_initializer='he_normal')(pool2)
    conv3 = BatchNormalization()(conv3)

    conv3 = Conv2D(64,
                   3,
                   activation='relu',
                   padding='same',
                   kernel_initializer='he_normal')(conv3)
    conv3 = BatchNormalization()(conv3)

    # se
    conv3 = SEModule(conv3, 8, 64)

    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(128,
                   3,
                   activation='relu',
                   padding='same',
                   kernel_initializer='he_normal')(pool3)
    conv4 = BatchNormalization()(conv4)

    conv4 = Conv2D(128,
                   3,
                   activation='relu',
                   padding='same',
                   kernel_initializer='he_normal')(conv4)
    conv4 = BatchNormalization()(conv4)

    # se
    conv4 = SEModule(conv4, 16, 128)

    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(256,
                   3,
                   activation='relu',
                   padding='same',
                   kernel_initializer='he_normal')(pool4)
    conv5 = BatchNormalization()(conv5)
    conv5 = Conv2D(256,
                   3,
                   activation='relu',
                   padding='same',
                   kernel_initializer='he_normal')(conv5)
    conv5 = BatchNormalization()(conv5)

    # se
    conv5 = SEModule(conv5, 16, 256)

    up6 = Conv2D(128,
                 2,
                 activation='relu',
                 padding='same',
                 kernel_initializer='he_normal')(UpSampling2D(size=(2,
                                                                    2))(conv5))
    up6 = BatchNormalization()(up6)

    merge6 = concatenate([conv4, up6], axis=3)
    conv6 = Conv2D(128,
                   3,
                   activation='relu',
                   padding='same',
                   kernel_initializer='he_normal')(merge6)
    conv6 = BatchNormalization()(conv6)

    conv6 = Conv2D(128,
                   3,
                   activation='relu',
                   padding='same',
                   kernel_initializer='he_normal')(conv6)
    conv6 = BatchNormalization()(conv6)

    # se
    conv6 = SEModule(conv6, 16, 128)

    up7 = Conv2D(64,
                 2,
                 activation='relu',
                 padding='same',
                 kernel_initializer='he_normal')(UpSampling2D(size=(2,
                                                                    2))(conv6))
    up7 = BatchNormalization()(up7)

    merge7 = concatenate([conv3, up7], axis=3)
    conv7 = Conv2D(64,
                   3,
                   activation='relu',
                   padding='same',
                   kernel_initializer='he_normal')(merge7)
    conv7 = BatchNormalization()(conv7)

    conv7 = Conv2D(64,
                   3,
                   activation='relu',
                   padding='same',
                   kernel_initializer='he_normal')(conv7)
    conv7 = BatchNormalization()(conv7)

    # se
    conv7 = SEModule(conv7, 8, 64)

    up8 = Conv2D(32,
                 2,
                 activation='relu',
                 padding='same',
                 kernel_initializer='he_normal')(UpSampling2D(size=(2,
                                                                    2))(conv7))
    up8 = BatchNormalization()(up8)

    merge8 = concatenate([conv2, up8], axis=3)
    conv8 = Conv2D(32,
                   3,
                   activation='relu',
                   padding='same',
                   kernel_initializer='he_normal')(merge8)
    conv8 = BatchNormalization()(conv8)

    conv8 = Conv2D(32,
                   3,
                   activation='relu',
                   padding='same',
                   kernel_initializer='he_normal')(conv8)
    conv8 = BatchNormalization()(conv8)

    # se
    conv8 = SEModule(conv8, 4, 32)

    up9 = Conv2D(16,
                 2,
                 activation='relu',
                 padding='same',
                 kernel_initializer='he_normal')(UpSampling2D(size=(2,
                                                                    2))(conv8))
    up9 = BatchNormalization()(up9)

    merge9 = concatenate([conv1, up9], axis=3)
    conv9 = Conv2D(16,
                   3,
                   activation='relu',
                   padding='same',
                   kernel_initializer='he_normal')(merge9)
    conv9 = BatchNormalization()(conv9)

    conv9 = Conv2D(16,
                   3,
                   activation='relu',
                   padding='same',
                   kernel_initializer='he_normal')(conv9)
    conv9 = BatchNormalization()(conv9)

    # se
    conv9 = SEModule(conv9, 2, 16)

    conv10 = Conv2D(nClasses, (3, 3), padding='same')(conv9)
    conv10 = BatchNormalization()(conv10)

    outputHeight = Model(inputs, conv10).output_shape[1]
    outputWidth = Model(inputs, conv10).output_shape[2]

    out = (Reshape((outputHeight * outputWidth, nClasses)))(conv10)
    out = Activation('softmax')(out)

    model = Model(inputs,out)
    model.outputHeight = outputHeight
    model.outputWidth = outputWidth

    return model

model1 = SEUnet(nClasses=5)
model1.summary()
x = model1.get_layer(index=-3).output

# folder=["Sprout","Black_smut","Rotten","Background"]
out0 = Conv2D(2, (1, 1), activation='softmax',name='sprout')(x)
out1 = Conv2D(2, (1, 1), activation='softmax',name='peeled')(x)
out2 = Conv2D(2, (1, 1), activation='softmax',name='rotten')(x)
out3 = Conv2D(2, (1, 1), activation='softmax',name='double')(x)
out4 = Conv2D(2, (1, 1), activation='softmax',name='background')(x)


model_new = Model(inputs = model1.input,outputs = [out0,out1,out2,out3,out4])
model_new.summary()
from segmentation_models.metrics import  f1_score,precision,recall
from segmentation_models.metrics import  f1_score
from tensorflow.keras.callbacks import TensorBoard
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

from tensorflow.keras.optimizers import Adam
model_name = 'onin_24th_jan_spplrobg'
date = '24thdec2024'
learning_rate = 0.001
# num_epochs = 20




# Initialize the optimizer
opt = Adam(lr=learning_rate)



# Define losses, metrics, and loss weights
losses = {"sprout": sm.losses.bce_jaccard_loss, "peeled": sm.losses.bce_jaccard_loss, "rotten": sm.losses.bce_jaccard_loss,"double": sm.losses.bce_jaccard_loss,"background": sm.losses.bce_jaccard_loss}
metrics = {"sprout": [iou_score, f1_score,precision,recall], "peeled": [iou_score, f1_score,precision,recall], "rotten": [iou_score, f1_score,precision,recall],"double": [iou_score, f1_score,precision,recall], "background": [iou_score, f1_score,precision,recall]}
loss_weights = {"sprout": 1.0, "peeled": 1.0, "rotten": 1.0, "double": 1.0, "background": 2.0}

print("[INFO] compiling model...")

# Define callbacks
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

LR_Reduce_callback = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=10, verbose=1, mode='auto')
model_ckpt1 = ModelCheckpoint(model_name+'_'+date+'.h5', monitor='loss', save_weights_only=False,save_best_only=True, period=1)
model_ckpt2 = ModelCheckpoint(model_name+'_'+date+'_weights.h5', monitor='loss', save_weights_only=True,save_best_only=True, period=1)
# reduce_rl_plateau = CustomReduceLRoP(patience=4, 
#                               verbose=1, 
#                               optim_lr=opt.learning_rate, 
#                               mode='auto',
#                               factor=0.1)
# Compile the model using the initialized optimizer
model_new.compile(optimizer=opt, loss=losses, metrics=metrics)
labels_dict = {
    "sprout": train_sprout,
    "peeled": train_peeled,
    "rotten": train_rotten,
    "double": train_double,
    "background": train_background
}
val_labels_dict = {
    "sprout": val_sprout,
    "peeled": val_peeled,
    "rotten": val_rotten,
    "double": val_double,
    "background": val_background
}
val_data = tf.data.Dataset.zip((val_images, val_labels_dict)) 
# Zip images and label dictionary into a single dataset
train_data = tf.data.Dataset.zip((train_images, labels_dict))

H = model_new.fit(train_data,
                  epochs=2,
                  validation_data=val_data,
                  callbacks=[tensorboard_callback, LR_Reduce_callback, model_ckpt1, model_ckpt2],
                  batch_size=16)


is there any issues in the data pipelines and data loader and training ...cuz accuracy is not being increased in this pipeline and please confirm me that is it like every time new batch of data is being uploaded or same batch is being repeated ....in every epochs the whole dtaasets should be repeated...where is the issue why the accuracy is not being increased even after 5k data provided

In [ ]:
import matplotlib.pyplot as plt

def visualize_data(images, masks):
    fig, axs = plt.subplots(1, 6, figsize=(15, 5))
    axs[0].imshow(images[0].numpy().astype("uint8"))
    axs[0].set_title("Image")
    
    mask_titles = ["Sprout", "Peeled", "Rotten", "Double", "Background"]
    
    for i in range(5):
        axs[i+1].imshow(masks[i][0].numpy().astype("uint8"), cmap="gray")
        axs[i+1].set_title(mask_titles[i])
    
    plt.show()

# Get a batch of data
for batch_images, batch_masks in train_data.take(1):
    visualize_data(batch_images, [batch_masks["sprout"], batch_masks["peeled"], batch_masks["rotten"], batch_masks["double"], batch_masks["background"]])
